# 💳 SmartSentry AML — Notebook 2: Transaction Generator
---
**Prerequisites:** Run Notebook 1 first, or load the CSVs from `./outputs/`.

**Purpose:** Generate the transactions table with:
- Legitimate transactions (normal behaviour)
- 5 fraud typologies injected
- Full PK/FK linkage to customers, accounts, devices, beneficiaries
- 30+ derived features including rolling velocity, time windows, and risk signals

**PK / FK schema for transactions:**

| Column | Type | Links to |
|---|---|---|
| `transaction_id` | PK | — |
| `customer_id` | FK | `customers.customer_id` |
| `sender_account_id` | FK | `accounts.account_id` |
| `receiver_account_id` | FK (nullable) | `accounts.account_id` — internal transfers only |
| `beneficiary_id` | FK (nullable) | `beneficiaries.beneficiary_id` — external transfers only |
| `device_id` | FK | `devices.device_id` |

> **NULL / XOR rule:** Exactly one of `receiver_account_id` / `beneficiary_id` is non-NULL per row.
> Internal transfer → `receiver_account_id` set. External transfer → `beneficiary_id` set.

In [1]:
# ── CELL 1: Imports ──────────────────────────────────────────────────────
import random
import warnings
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

OUTPUT_DIR = Path("./outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

print("✅ Libraries loaded")

✅ Libraries loaded


In [3]:
# ── CELL 2: Configuration ────────────────────────────────────────────────
# All parameters for transaction generation live here.

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# ─── Simulation Window ────────────────────────────────────────────────────
START_DATE = datetime(2025, 9, 1)
END_DATE   = datetime(2025, 12, 31)
DATE_RANGE_DAYS = (END_DATE - START_DATE).days   # 121 days

# ─── Transaction Volume ───────────────────────────────────────────────────
TARGET_FRAUD_RATE         = 0.10   # 4% overall fraud prevalence
AVG_TXNS_PER_ACCOUNT_PER_DAY = 50 / DATE_RANGE_DAYS  # ~0.413 tx/account/day 
# Poisson lambda scales linearly with the simulation window so that
# a longer date range always produces proportionally more transactions.
AVG_TXNS_PER_ACCOUNT = AVG_TXNS_PER_ACCOUNT_PER_DAY * DATE_RANGE_DAYS

# ─── Dormant Account Injection ───────────────────────────────────────────
# Windows expressed as FRACTIONS of DATE_RANGE_DAYS so they scale
# automatically when START_DATE / END_DATE change.
#   first cluster  : first 20% of the window  (gap starts here)
#   second cluster : from 55% of the window to end  (reactivation)
#   guaranteed gap : ~35% of DATE_RANGE_DAYS (≥30 days for any window >86 days)
DORMANT_ACCOUNT_FRAC  = 0.05
_dorm_split_end   = max(25, int(DATE_RANGE_DAYS * 0.20))  # end of first cluster
_dorm_second_start = max(_dorm_split_end + 35,             # enforce minimum 35-day gap
                          int(DATE_RANGE_DAYS * 0.55))     # start of reactivation cluster
DORMANT_FIRST_WINDOW  = (0, _dorm_split_end)
DORMANT_SECOND_WINDOW = (_dorm_second_start, DATE_RANGE_DAYS)
DORMANT_FIRST_TXNS    = (3, 8)    # tx count in first cluster (absolute — short burst)
DORMANT_SECOND_TXNS   = (5, 15)   # tx count in second cluster (reactivation burst)

# ─── Channels & Transaction Types ─────────────────────────────────────────
CHANNELS      = ["mobile", "web", "branch", "atm"]
CHANNEL_PROBS = [0.50,     0.30,  0.10,    0.10]

# Transaction types available per channel
TXN_TYPES_BY_CHANNEL = {
    "mobile" : ["UPI", "IMPS", "wallet_transfer"],
    "web"    : ["NEFT", "RTGS", "online_transfer"],
    "branch" : ["cash_deposit", "cash_withdrawal", "DD"],
    "atm"    : ["cash_withdrawal", "balance_enquiry"],
}

# These transaction types involve physical cash → cash_flag = 1
CASH_TXN_TYPES = {"cash_deposit", "cash_withdrawal", "DD"}

# ─── Amount Parameters by KYC Level ──────────────────────────────────────
AMOUNT_BY_KYC = {
    "low"    : {"mean": 7.0, "sigma": 1.0},
    "medium" : {"mean": 8.0, "sigma": 1.0},
    "high"   : {"mean": 9.5, "sigma": 1.0},
}
AMOUNT_DEFAULT = {"mean": 8.0, "sigma": 1.0}

# 40% of transactions are internal (account to account)
# 60% are external (account to beneficiary)
INTERNAL_TRANSFER_PROB = 0.40

# ─── Transaction Timing ───────────────────────────────────────────────────
# Hour-of-day weights (index 0=midnight … 23=11pm)
HOUR_WEIGHTS_NORMAL = [1,1,1,1,1,1,2,3,5,6,6,6,5,6,6,6,5,5,4,4,3,2,2,1]
HOUR_WEIGHTS_FRAUD  = [4,4,4,4,3,2,1,1,1,1,1,1,1,1,1,1,2,2,3,3,4,4,4,4]

# ─── Fraud Typology Volumes ────────────────────────────────────────────────
# All day_min / day_max values are expressed as integer day offsets from
# START_DATE. They are computed from DATE_RANGE_DAYS fractions so that
# fraud clusters scale proportionally when the simulation window changes.
# Absolute counts (num_rings, num_chains, count, num_groups) scale by
# DATE_RANGE_DAYS / 121 so volume grows with a longer window.
_scale = DATE_RANGE_DAYS / 121   # 1.0 at original 121-day window

FRAUD_VOLUMES = {
    "mule_ring": {
        "num_rings"        : max(1, int(60  * _scale)),
        "accounts_per_ring": 10,
        "txns_per_account" : 8,
        # Days 50%–99% of window — mid-to-late activity
        "day_min"          : int(DATE_RANGE_DAYS * 0.50),
        "day_max"          : int(DATE_RANGE_DAYS * 0.99),
        "amount_mean"      : 9.5,   "amount_sigma": 0.5,
        "channels"         : ["mobile", "web"],
    },
    "layering": {
        "num_chains"       : max(1, int(900 * _scale)),
        "chain_length"     : 3,
        # Days 60%–99% — later in the window (funds have been placed)
        "day_min"          : int(DATE_RANGE_DAYS * 0.60),
        "day_max"          : int(DATE_RANGE_DAYS * 0.99),
        "hop_min_mins"     : 5,     "hop_max_mins": 30,
        "amount_mean_base" : 9.0,   "amount_mean_decay": 0.2,
        "amount_sigma"     : 0.8,
        "channels"         : ["mobile", "web"],
    },
    "ATO": {
        "count"            : max(1, int(2000 * _scale)),
        # Days 70%–99% — ATO typically exploits accounts once established
        "day_min"          : int(DATE_RANGE_DAYS * 0.70),
        "day_max"          : int(DATE_RANGE_DAYS * 0.99),
        "amount_mean"      : 10.5,  "amount_sigma": 0.8,
        "channels"         : ["mobile", "web"],
    },
    "smurfing": {
        "num_groups"       : max(1, int(400 * _scale)),
        "min_txns"         : 4,     "max_txns": 8,
        "amount_min"       : 8500,  "amount_max": 9999,
        "day_window"       : 2,
        "channels"         : ["branch", "atm"],
        "force_cash_flag"  : True,
    },
    "identity_fraud": {
        "count"            : max(1, int(800 * _scale)),
        "max_account_days" : 60,
        # Days 0–25% — identity fraud hits immediately after account opening
        "day_min"          : 0,
        "day_max"          : int(DATE_RANGE_DAYS * 0.25),
        "amount_mean"      : 11.0,  "amount_sigma": 0.6,
        "channels"         : ["mobile", "web"],
    },
    # ── Dormant Account Fraud ─────────────────────────────────────────────
    # Three sub-typologies that specifically TARGET dormant accounts.
    # All fraud timestamps fall within DORMANT_SECOND_WINDOW (day 65+) so
    # that dormancy_flag=1 will fire for these rows in the velocity engine.
    "dormant_ATO": {
        # Account Takeover: fraudster hijacks the dormant account using a
        # new/unrecognised device, immediately drains funds to a high-risk bene.
        # Key signals: new device + dormancy_flag=1 + large amount + night hour.
        # Targets ~60% of dormant accounts (those with higher balances).
        "frac_dormant_targeted": 0.60,  # fraction of dormant_account_set to hit
        "txns_per_account"     : (2, 5),  # rapid burst (min, max)
        "amount_mean"          : 11.0,   "amount_sigma": 0.7,  # large amounts
        "channels"             : ["mobile", "web"],
        # device: always a DIFFERENT device (not the account's home device)
        # timing: within DORMANT_SECOND_WINDOW so dormancy_flag fires
    },
    "dormant_smurfing": {
        # Structuring via dormant accounts: fraudster reactivates account and
        # makes multiple cash transactions just below ₹10k threshold.
        # Combined dormancy + structuring is a strong AML red flag.
        # Targets ~25% of dormant accounts.
        "frac_dormant_targeted": 0.25,
        "min_txns"             : 3,      "max_txns": 6,
        "amount_min"           : 8500,   "amount_max": 9999,
        "channels"             : ["branch", "atm"],
        "force_cash_flag"      : True,
        "day_window"           : 3,  # spread over 3 days within reactivation window
    },
    "dormant_to_offshore": {
        # Dormant account suddenly wires large amount offshore: classic
        # reactivation-for-laundering pattern. Funds may have been placed
        # earlier; this is the integration/extraction step.
        # Targets ~30% of dormant accounts.
        "frac_dormant_targeted": 0.30,
        "txns_per_account"     : (1, 3),
        "amount_mean"          : 11.5,   "amount_sigma": 0.5,  # very large
        "channels"             : ["web", "mobile"],
        # beneficiary: always offshore or crypto (high-risk pool)
    },
}


# ─── IP / Geo Config ─────────────────────────────────────────────────────────
IP_HOME_FRAC         = 0.70   # fraction of txns using home IP
IP_ROAM_FRAC         = 0.20   # fraction using roaming IP
# remaining 10% → high-risk proxy IP (always forced when vpn_flag=1)
IP_HOME_BASE_SCORE   = 0.05
IP_ROAM_BASE_SCORE   = 0.30
IP_RISK_BASE_SCORE   = 0.80
HIGH_RISK_IP_PREFIXES = ["185.220","45.142","198.96","176.10","162.247",
                          "91.108","77.247","104.244","194.165","185.107"]

# ─── Fraud Intensity Score (FIS) weights ─────────────────────────────────────
FIS_W_RULE    = 0.5    # weight on weighted_rule_score
FIS_W_LABEL   = 0.3    # weight on label × 10 (known fraud bonus)
FIS_W_BALANCE = 0.2    # weight on min(amount_to_balance_ratio, 10)
FIS_SCALE     = 100    # cap — score bounded to 0–100

print("✅ Transaction configuration loaded")
print(f"   Simulation window    : {START_DATE.date()} → {END_DATE.date()} ({DATE_RANGE_DAYS} days)")
print(f"   Target fraud rate    : {TARGET_FRAUD_RATE:.0%}")
print(f"   Dormant account %    : {DORMANT_ACCOUNT_FRAC:.0%} of accounts forced dormant")
print(f"   Scale factor (_scale): {_scale:.3f}x  (1.0 = original 121-day window)")
print(f"   AVG_TXNS_PER_ACCOUNT : {AVG_TXNS_PER_ACCOUNT:.1f}  (= {AVG_TXNS_PER_ACCOUNT_PER_DAY:.4f}/day × {DATE_RANGE_DAYS} days)")
print(f"   Dormant first window : days {DORMANT_FIRST_WINDOW[0]}–{DORMANT_FIRST_WINDOW[1]}")
print(f"   Dormant second window: days {DORMANT_SECOND_WINDOW[0]}–{DORMANT_SECOND_WINDOW[1]} "
      f"(gap = {DORMANT_SECOND_WINDOW[0] - DORMANT_FIRST_WINDOW[1]} days)")
print()
print("   Fraud volumes (scaled):")
for name, cfg in FRAUD_VOLUMES.items():
    vol = (cfg.get('num_rings', cfg.get('num_chains', cfg.get('count', cfg.get('num_groups', '—')))))
    d_min = cfg.get('day_min', '—')
    d_max = cfg.get('day_max', '—')
    print(f"     {name:<22} volume={vol}  days {d_min}–{d_max}")

✅ Transaction configuration loaded
   Simulation window    : 2025-09-01 → 2025-12-31 (121 days)
   Target fraud rate    : 10%
   Dormant account %    : 5% of accounts forced dormant
   Scale factor (_scale): 1.000x  (1.0 = original 121-day window)
   AVG_TXNS_PER_ACCOUNT : 50.0  (= 0.4132/day × 121 days)
   Dormant first window : days 0–25
   Dormant second window: days 66–121 (gap = 41 days)

   Fraud volumes (scaled):
     mule_ring              volume=60  days 60–119
     layering               volume=900  days 72–119
     ATO                    volume=2000  days 84–119
     smurfing               volume=400  days —–—
     identity_fraud         volume=800  days 0–30
     dormant_ATO            volume=—  days —–—
     dormant_smurfing       volume=—  days —–—
     dormant_to_offshore    volume=—  days —–—


In [4]:
# ── CELL 3: Load Reference Tables ────────────────────────────────────────
# Load the four tables generated in Notebook 1.
# If you ran Notebook 1 in the same session, these are already in memory —
# this cell reloads them from CSV so the notebook is self-contained.

customers     = pd.read_csv(OUTPUT_DIR / "customers.csv")
accounts      = pd.read_csv(OUTPUT_DIR / "accounts.csv")
devices       = pd.read_csv(OUTPUT_DIR / "devices.csv")
beneficiaries = pd.read_csv(OUTPUT_DIR / "beneficiaries.csv")

# Rebuild lookup dicts
device_list = devices["device_id"].tolist()
account_device_map = {acc: random.choice(device_list) for acc in accounts["account_id"]}

bene_list = beneficiaries["beneficiary_id"].tolist()
account_beneficiaries_map = {
    acc: random.sample(bene_list, k=random.randint(2, 5))
    for acc in accounts["account_id"]
}

high_risk_mask = (
    beneficiaries["beneficiary_type"].isin({"crypto", "offshore"}) |
    (beneficiaries["beneficiary_country_risk"] == "high")
)
high_risk_bene_pool = beneficiaries.loc[high_risk_mask, "beneficiary_id"].tolist()

all_account_ids = accounts["account_id"].tolist()

# ── FIX: Select 5% dormant accounts upfront ───────────────────────────────
# We exclude these from the main Poisson loop and generate them separately
# with a guaranteed 30+ day gap between transaction clusters.
n_dormant = max(1, int(len(all_account_ids) * DORMANT_ACCOUNT_FRAC))
dormant_account_set = set(random.sample(all_account_ids, n_dormant))

print("✅ Reference tables loaded")
print(f"   customers: {len(customers):,} | accounts: {len(accounts):,} | "
      f"devices: {len(devices):,} | beneficiaries: {len(beneficiaries):,}")
print(f"   high_risk_bene_pool: {len(high_risk_bene_pool):,}")
print(f"   dormant accounts   : {len(dormant_account_set):,} ({len(dormant_account_set)/len(all_account_ids):.1%})")
# ── Pull geo seed + shared identity from accounts ─────────────────────────────
# These were denormalised from customers into accounts in Notebook 1.
# We build a per-customer lookup here so the IP/geo cell (8b) can access
# home coordinates by customer_id without re-joining on every row.
if "home_lat" in accounts.columns:
    cust_geo_map = (
        accounts[["customer_id","home_lat","home_lon"]]
        .drop_duplicates("customer_id")
        .set_index("customer_id")
    )
else:
    cust_geo_map = None   # graceful fallback if Notebook 1 was not updated



✅ Reference tables loaded
   customers: 5,000 | accounts: 8,500 | devices: 4,500 | beneficiaries: 6,000
   high_risk_bene_pool: 1,099
   dormant accounts   : 425 (5.0%)


In [5]:
# ── CELL 4: Shared Helper Functions ──────────────────────────────────────

def weighted_hour(fraud=False):
    """Sample hour (0–23) weighted by transaction behaviour profile."""
    weights = HOUR_WEIGHTS_FRAUD if fraud else HOUR_WEIGHTS_NORMAL
    return random.choices(range(24), weights=weights)[0]

def random_ts(fraud=False, day_min=0, day_max=None):
    """Random datetime within simulation window."""
    if day_max is None:
        day_max = DATE_RANGE_DAYS
    day_max = min(day_max, DATE_RANGE_DAYS)  # clamp to simulation end
    return START_DATE + timedelta(
        days=random.randint(day_min, day_max),
        hours=weighted_hour(fraud),
        minutes=random.randint(0, 59),
        seconds=random.randint(0, 59),
    )

def get_txn_type(channel):
    """Random transaction type for a given channel."""
    return random.choice(TXN_TYPES_BY_CHANNEL.get(channel, ["online_transfer"]))

def is_cash(txn_type):
    """1 if transaction involves physical cash, else 0."""
    return int(txn_type in CASH_TXN_TYPES)

# Canonical column order — flow tracing fields inserted after cash_flag
TXN_COLS = [
    "transaction_id",
    "customer_id",
    "sender_account_id",
    "receiver_account_id",
    "beneficiary_id",
    "device_id",
    "timestamp",
    "amount",
    "channel",
    "debit_credit",
    "transaction_type",
    "cash_flag",            # index 11
    # ── Flow Tracing (NULL for non-chain/ring typologies) ─────────────────
    "synthetic_flow_id",    # index 12 — FLOW_XXXXX or None
    "flow_depth",           # index 13 — total hops in flow or None
    "hop_number",           # index 14 — 1-indexed position in flow or None
    "time_since_origin_ts", # index 15 — raw origin datetime, converted later
    # ── Labels ────────────────────────────────────────────────────────────
    "fraud_type",           # index 16
    "label",                # index 17
]

# Pre-compute index lookups once so Cell 6 never uses hardcoded integers
_FRAUD_TYPE_IDX = TXN_COLS.index("fraud_type")   # 16
_CASH_FLAG_IDX  = TXN_COLS.index("cash_flag")    # 11

def build_row(txn_id, customer_id, sender_account_id, device_id,
              ts, amount, channel, fraud_type, label,
              receiver_account_id=None, beneficiary_id=None,
              synthetic_flow_id=None, flow_depth=None,
              hop_number=None, time_since_origin_ts=None,
              force_cash=False):
    """
    Build one canonical transaction row aligned to TXN_COLS.
    Flow tracing fields default to None (NULL) for non-flow typologies.
    force_cash=True overrides cash_flag to 1 (used by smurfing typologies).
    """
    txn_type = get_txn_type(channel)
    cash     = 1 if force_cash else is_cash(txn_type)
    return [
        f"T{txn_id}",          # 0  transaction_id
        customer_id,            # 1  customer_id
        sender_account_id,      # 2  sender_account_id
        receiver_account_id,    # 3  receiver_account_id
        beneficiary_id,         # 4  beneficiary_id
        device_id,              # 5  device_id
        ts,                     # 6  timestamp
        round(amount, 2),       # 7  amount
        channel,                # 8  channel
        "debit",                # 9  debit_credit
        txn_type,               # 10 transaction_type
        cash,                   # 11 cash_flag
        synthetic_flow_id,      # 12 synthetic_flow_id
        flow_depth,             # 13 flow_depth
        hop_number,             # 14 hop_number
        time_since_origin_ts,   # 15 time_since_origin_ts
        fraud_type,             # 16 fraud_type
        label,                  # 17 label
    ]

print("✅ Helper functions defined")
print(f"   TXN_COLS ({len(TXN_COLS)} columns): {TXN_COLS}")

✅ Helper functions defined
   TXN_COLS (18 columns): ['transaction_id', 'customer_id', 'sender_account_id', 'receiver_account_id', 'beneficiary_id', 'device_id', 'timestamp', 'amount', 'channel', 'debit_credit', 'transaction_type', 'cash_flag', 'synthetic_flow_id', 'flow_depth', 'hop_number', 'time_since_origin_ts', 'fraud_type', 'label']


In [6]:
# ── CELL 5: Generate Legitimate Transactions ─────────────────────────────
# Each account sends ~50 transactions (Poisson distributed).
# 40% are internal (account-to-account), 60% external (to beneficiary).
#
# FIX: Dormant accounts (5%) are generated separately with two transaction
# clusters and a guaranteed 30+ day gap between them. All other accounts
# receive uniformly distributed timestamps as before.

acc_meta  = accounts.set_index("account_id")
legit_rows = []
txn_id     = 0

def _add_legit_txn(acc, cust, device, kyc, day_min, day_max, txn_id):
    """Helper: build one legit transaction row for a given time window."""
    params = AMOUNT_BY_KYC.get(kyc, AMOUNT_DEFAULT)
    ts     = random_ts(fraud=False, day_min=day_min, day_max=day_max)
    amount = np.random.lognormal(mean=params["mean"], sigma=params["sigma"])
    ch     = random.choices(CHANNELS, weights=CHANNEL_PROBS)[0]

    if random.random() < INTERNAL_TRANSFER_PROB:
        return build_row(
            txn_id, cust, acc, device, ts, amount, ch, "normal", 0,
            receiver_account_id=random.choice(all_account_ids),
            beneficiary_id=None,
        )
    else:
        return build_row(
            txn_id, cust, acc, device, ts, amount, ch, "normal", 0,
            receiver_account_id=None,
            beneficiary_id=random.choice(account_beneficiaries_map[acc]),
        )

dormant_count_total = 0  # track how many dormant rows we inject

for acc in accounts["account_id"]:
    kyc    = acc_meta.loc[acc, "kyc_level"]
    cust   = acc_meta.loc[acc, "customer_id"]
    device = account_device_map[acc]

    if acc in dormant_account_set:
        # ── DORMANT ACCOUNT: two clusters with a 40+ day gap ──────────────
        # Cluster 1: a few transactions early in the simulation
        n_first  = random.randint(*DORMANT_FIRST_TXNS)
        n_second = random.randint(*DORMANT_SECOND_TXNS)

        for _ in range(n_first):
            row = _add_legit_txn(acc, cust, device, kyc,
                                  DORMANT_FIRST_WINDOW[0],
                                  DORMANT_FIRST_WINDOW[1],
                                  txn_id)
            legit_rows.append(row)
            txn_id += 1

        # Cluster 2: reactivation burst (this is where dormancy_flag fires)
        for _ in range(n_second):
            row = _add_legit_txn(acc, cust, device, kyc,
                                  DORMANT_SECOND_WINDOW[0],
                                  DORMANT_SECOND_WINDOW[1],
                                  txn_id)
            legit_rows.append(row)
            txn_id += 1

        dormant_count_total += n_first + n_second

    else:
        # ── NORMAL ACCOUNT: uniform Poisson transactions ──────────────────
        n_txns = np.random.poisson(AVG_TXNS_PER_ACCOUNT)
        for _ in range(n_txns):
            row = _add_legit_txn(acc, cust, device, kyc, 0, DATE_RANGE_DAYS, txn_id)
            legit_rows.append(row)
            txn_id += 1

legit_df = pd.DataFrame(legit_rows, columns=TXN_COLS)

print(f"✅ Legitimate transactions: {len(legit_df):,} rows")
print(f"   Normal accounts        : {len(all_account_ids) - len(dormant_account_set):,}")
print(f"   Dormant accounts forced: {len(dormant_account_set):,} "
      f"({len(dormant_account_set)/len(all_account_ids):.1%} of accounts, {dormant_count_total:,} rows)")
print(f"   Internal transfers     : {legit_df['receiver_account_id'].notna().sum():,} "
      f"({legit_df['receiver_account_id'].notna().mean()*100:.1f}%)")
print(f"   External transfers     : {legit_df['beneficiary_id'].notna().sum():,} "
      f"({legit_df['beneficiary_id'].notna().mean()*100:.1f}%)")
print()
legit_df.head(3)

✅ Legitimate transactions: 411,331 rows
   Normal accounts        : 8,075
   Dormant accounts forced: 425 (5.0% of accounts, 6,642 rows)
   Internal transfers     : 164,741 (40.1%)
   External transfers     : 246,590 (59.9%)



,transaction_id,customer_id,sender_account_id,receiver_account_id,beneficiary_id,device_id,timestamp,amount,channel,debit_credit,transaction_type,cash_flag,synthetic_flow_id,flow_depth,hop_number,time_since_origin_ts,fraud_type,label
0,T0,C742,A0,None,B3294,D912,2025-10-15 06:12:54,2095.80,mobile,debit,UPI,0,None,None,None,None,normal,0
1,T1,C742,A0,None,B3294,D912,2025-09-09 17:34:06,5029.27,mobile,debit,UPI,0,None,None,None,None,normal,0
2,T2,C742,A0,None,B1422,D912,2025-12-15 20:30:55,867.70,atm,debit,balance_enquiry,0,None,None,None,None,normal,0


In [7]:
# ── CELL 6: Inject Fraud Typologies ──────────────────────────────────────────
# Injects all 8 fraud typologies into fraud_rows using build_row().
#
# Flow tracing (synthetic_flow_id, flow_depth, hop_number, time_since_origin_ts):
#   ✅ mule_ring   — one FLOW_ID per ring; hop_number = position in ring
#   ✅ layering    — one FLOW_ID per chain; hop_number = 1→chain_length
#   ✗  ATO, smurfing, identity_fraud, dormant_*  — flow fields = None (not chains)
#
# cash_flag override: smurfing typologies pass force_cash=True to build_row().
#   This is safe — no index arithmetic, build_row() handles it internally.
#
# fraud_type filtering uses _FRAUD_TYPE_IDX (= TXN_COLS.index("fraud_type"))
#   to avoid hardcoded integers that drift when columns are added/removed.

acc_meta_fraud = accounts.set_index("account_id")
all_accs       = accounts["account_id"].tolist()
all_devs       = devices["device_id"].tolist()
fraud_rows     = []

# ── 6A: MULE RINGS ────────────────────────────────────────────────────────────
# Flow: each ring → one FLOW_ID shared by all accounts in that ring.
# hop_number = account position within the ring (1-indexed).
# time_since_origin_ts = ring start timestamp (used later to compute seconds elapsed).
cfg = FRAUD_VOLUMES["mule_ring"]
_mule_flow_counter = 0

for _ in range(cfg["num_rings"]):
    mule_accs      = random.sample(all_accs, cfg["accounts_per_ring"])
    shared_dev     = random.choice(all_devs)
    exit_bene      = random.choice(high_risk_bene_pool)
    flow_id        = f"FLOW_{_mule_flow_counter:05d}"
    ring_depth     = len(mule_accs)                        # = accounts_per_ring
    ring_origin_ts = random_ts(True, cfg["day_min"], cfg["day_max"])
    _mule_flow_counter += 1

    for hop_i, acc in enumerate(mule_accs, start=1):
        cust = acc_meta_fraud.loc[acc, "customer_id"]
        for _ in range(cfg["txns_per_account"]):
            ts  = random_ts(True, cfg["day_min"], cfg["day_max"])
            amt = np.random.lognormal(cfg["amount_mean"], cfg["amount_sigma"])
            ch  = random.choice(cfg["channels"])
            fraud_rows.append(build_row(
                txn_id, cust, acc, shared_dev, ts, amt, ch,
                "mule_ring", 1,
                receiver_account_id=None, beneficiary_id=exit_bene,
                synthetic_flow_id=flow_id, flow_depth=ring_depth,
                hop_number=hop_i, time_since_origin_ts=ring_origin_ts,
            ))
            txn_id += 1

_mr_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == "mule_ring")
print(f"  Mule rings         : {_mr_count:>6,} rows  ({cfg['num_rings']} rings × {cfg['accounts_per_ring']} accs × {cfg['txns_per_account']} txns)")

# ── 6B: LAYERING CHAINS ───────────────────────────────────────────────────────
# Flow: each chain → one FLOW_ID (offset 10 000 from mule ring IDs).
# hop_number = 1 (origin account) through chain_length (final account).
# time_since_origin_ts = base_ts (the timestamp of hop 1 for the whole chain).
# Amount decays each hop: lognormal mean decreases by amount_mean_decay per step.
cfg = FRAUD_VOLUMES["layering"]
_layer_flow_counter = 10_000   # namespace offset — layering IDs never collide with mule ring IDs

for _ in range(cfg["num_chains"]):
    chain   = random.sample(all_accs, cfg["chain_length"])
    chain   = chain + [chain[0]]    # close loop back to origin
    base_ts = random_ts(True, cfg["day_min"], cfg["day_max"])
    flow_id = f"FLOW_{_layer_flow_counter:05d}"
    chain_depth = cfg["chain_length"]
    _layer_flow_counter += 1

    for i in range(cfg["chain_length"]):
        sender   = chain[i]
        receiver = chain[i + 1]
        cust     = acc_meta_fraud.loc[sender, "customer_id"]
        device   = account_device_map[sender]
        ts       = base_ts + timedelta(
                       minutes=random.randint(cfg["hop_min_mins"], cfg["hop_max_mins"]) * (i + 1)
                   )
        amt = np.random.lognormal(
            cfg["amount_mean_base"] - i * cfg["amount_mean_decay"],
            cfg["amount_sigma"]
        )
        ch = random.choice(cfg["channels"])
        fraud_rows.append(build_row(
            txn_id, cust, sender, device, ts, amt, ch,
            "layering", 1,
            receiver_account_id=receiver, beneficiary_id=None,
            synthetic_flow_id=flow_id, flow_depth=chain_depth,
            hop_number=i + 1, time_since_origin_ts=base_ts,
        ))
        txn_id += 1

_ly_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == "layering")
print(f"  Layering chains    : {_ly_count:>6,} rows  ({cfg['num_chains']} chains × {cfg['chain_length']} hops)")

# ── 6C: ACCOUNT TAKEOVER (ATO) ────────────────────────────────────────────────
# Not a flow — single extraction event per occurrence.
# Key signal: new_dev intentionally differs from account home device.
# Flow fields all default to None.
cfg = FRAUD_VOLUMES["ATO"]

for _ in range(cfg["count"]):
    acc       = random.choice(all_accs)
    cust      = acc_meta_fraud.loc[acc, "customer_id"]
    new_dev   = random.choice(all_devs)          # unrecognised device = ATO signal
    exit_bene = random.choice(high_risk_bene_pool)
    ts        = random_ts(True, cfg["day_min"], cfg["day_max"])
    amt       = np.random.lognormal(cfg["amount_mean"], cfg["amount_sigma"])
    ch        = random.choice(cfg["channels"])
    fraud_rows.append(build_row(
        txn_id, cust, acc, new_dev, ts, amt, ch,
        "ATO", 1,
        receiver_account_id=None, beneficiary_id=exit_bene,
        # flow fields → None: ATO is a point event, not a structured chain
    ))
    txn_id += 1

_ato_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == "ATO")
print(f"  ATO                : {_ato_count:>6,} rows")

# ── 6D: SMURFING / STRUCTURING ────────────────────────────────────────────────
# Not a flow — burst of cash transactions within a 2-day window.
# force_cash=True sets cash_flag=1 safely via build_row(); no index arithmetic.
# Flow fields all default to None.
cfg = FRAUD_VOLUMES["smurfing"]

for _ in range(cfg["num_groups"]):
    acc       = random.choice(all_accs)
    cust      = acc_meta_fraud.loc[acc, "customer_id"]
    device    = account_device_map[acc]
    exit_bene = random.choice(high_risk_bene_pool)
    base_day  = random.randint(0, DATE_RANGE_DAYS - cfg["day_window"] - 1)

    for _ in range(random.randint(cfg["min_txns"], cfg["max_txns"])):
        ts  = random_ts(True, day_min=base_day, day_max=base_day + cfg["day_window"])
        amt = round(random.uniform(cfg["amount_min"], cfg["amount_max"]), 2)
        ch  = random.choice(cfg["channels"])
        fraud_rows.append(build_row(
            txn_id, cust, acc, device, ts, amt, ch,
            "smurfing", 1,
            receiver_account_id=None, beneficiary_id=exit_bene,
            force_cash=True,     # ← cash_flag=1 guaranteed; no list-index tricks
        ))
        txn_id += 1

_sm_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == "smurfing")
print(f"  Smurfing           : {_sm_count:>6,} rows  ({cfg['num_groups']} groups)")

# ── 6E: IDENTITY FRAUD ────────────────────────────────────────────────────────
# Targets newly opened accounts (< max_account_days old).
# Not a flow — single extraction event.  Flow fields all None.
cfg      = FRAUD_VOLUMES["identity_fraud"]
new_accs = accounts.loc[
    accounts["account_open_days"] < cfg["max_account_days"], "account_id"
].tolist()
sample_n = min(cfg["count"], len(new_accs))

for acc in random.sample(new_accs, sample_n):
    cust      = acc_meta_fraud.loc[acc, "customer_id"]
    device    = account_device_map[acc]
    exit_bene = random.choice(high_risk_bene_pool)
    ts        = random_ts(True, cfg["day_min"], cfg["day_max"])
    amt       = np.random.lognormal(cfg["amount_mean"], cfg["amount_sigma"])
    ch        = random.choice(cfg["channels"])
    fraud_rows.append(build_row(
        txn_id, cust, acc, device, ts, amt, ch,
        "identity_fraud", 1,
        receiver_account_id=None, beneficiary_id=exit_bene,
    ))
    txn_id += 1

_id_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == "identity_fraud")
print(f"  Identity fraud     : {_id_count:>6,} rows")

# ── 6F: DORMANT ACCOUNT FRAUD ─────────────────────────────────────────────────
# Three sub-typologies, all exclusively targeting dormant_account_set.
# ALL timestamps fall within DORMANT_SECOND_WINDOW → dormancy_flag=1 fires.
# None of these are structured flows → flow fields all None.
# cash_flag override handled via force_cash=True in smurfing sub-type.

dormant_accs_list = list(dormant_account_set)

# ── 6F-i: Dormant ATO ────────────────────────────────────────────────────────
# Key signals: new_dev ≠ home device, large amount, dormancy_flag=1, night.
cfg   = FRAUD_VOLUMES["dormant_ATO"]
n_ato = max(1, int(len(dormant_accs_list) * cfg["frac_dormant_targeted"]))
ato_targets = random.sample(dormant_accs_list, n_ato)

for acc in ato_targets:
    cust       = acc_meta_fraud.loc[acc, "customer_id"]
    home_dev   = account_device_map[acc]
    other_devs = [d for d in all_devs if d != home_dev]
    new_dev    = random.choice(other_devs)    # unrecognised device = core ATO signal
    exit_bene  = random.choice(high_risk_bene_pool)
    n_txns     = random.randint(*cfg["txns_per_account"])
    base_day   = random.randint(DORMANT_SECOND_WINDOW[0], DORMANT_SECOND_WINDOW[1] - 1)

    for _ in range(n_txns):
        ts  = random_ts(True, day_min=base_day,
                        day_max=min(base_day + 1, DORMANT_SECOND_WINDOW[1]))
        amt = np.random.lognormal(cfg["amount_mean"], cfg["amount_sigma"])
        ch  = random.choice(cfg["channels"])
        fraud_rows.append(build_row(
            txn_id, cust, acc, new_dev, ts, amt, ch,
            "dormant_ATO", 1,
            receiver_account_id=None, beneficiary_id=exit_bene,
        ))
        txn_id += 1

_dato_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == "dormant_ATO")
print(f"  Dormant ATO        : {_dato_count:>6,} rows  ({n_ato} accounts targeted)")

# ── 6F-ii: Dormant Smurfing ───────────────────────────────────────────────────
# Reactivated dormant account used for cash structuring below ₹10k threshold.
# force_cash=True — physical cash involved regardless of txn_type.
cfg     = FRAUD_VOLUMES["dormant_smurfing"]
n_smurf = max(1, int(len(dormant_accs_list) * cfg["frac_dormant_targeted"]))
smurf_targets = random.sample(dormant_accs_list, n_smurf)

for acc in smurf_targets:
    cust      = acc_meta_fraud.loc[acc, "customer_id"]
    device    = account_device_map[acc]    # home device (less obvious than ATO)
    exit_bene = random.choice(high_risk_bene_pool)
    base_day  = random.randint(
        DORMANT_SECOND_WINDOW[0],
        DORMANT_SECOND_WINDOW[1] - cfg["day_window"] - 1
    )
    n_txns = random.randint(cfg["min_txns"], cfg["max_txns"])

    for _ in range(n_txns):
        ts  = random_ts(True, day_min=base_day, day_max=base_day + cfg["day_window"])
        amt = round(random.uniform(cfg["amount_min"], cfg["amount_max"]), 2)
        ch  = random.choice(cfg["channels"])
        fraud_rows.append(build_row(
            txn_id, cust, acc, device, ts, amt, ch,
            "dormant_smurfing", 1,
            receiver_account_id=None, beneficiary_id=exit_bene,
            force_cash=True,     # cash_flag=1 guaranteed via build_row
        ))
        txn_id += 1

_dsm_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == "dormant_smurfing")
print(f"  Dormant smurfing   : {_dsm_count:>6,} rows  ({n_smurf} accounts targeted)")

# ── 6F-iii: Dormant-to-Offshore ───────────────────────────────────────────────
# Large single wire from reactivated dormant account to offshore/crypto exit node.
# Integration step of placement→layering→integration cycle.
cfg        = FRAUD_VOLUMES["dormant_to_offshore"]
n_offshore = max(1, int(len(dormant_accs_list) * cfg["frac_dormant_targeted"]))
offshore_targets = random.sample(dormant_accs_list, n_offshore)

offshore_crypto_pool = beneficiaries.loc[
    beneficiaries["beneficiary_type"].isin({"crypto", "offshore"}),
    "beneficiary_id"
].tolist()
exit_pool = offshore_crypto_pool if offshore_crypto_pool else high_risk_bene_pool

for acc in offshore_targets:
    cust     = acc_meta_fraud.loc[acc, "customer_id"]
    device   = account_device_map[acc]
    n_txns   = random.randint(*cfg["txns_per_account"])
    base_day = random.randint(DORMANT_SECOND_WINDOW[0], DORMANT_SECOND_WINDOW[1] - 2)

    for _ in range(n_txns):
        exit_bene = random.choice(exit_pool)
        ts  = random_ts(True, day_min=base_day,
                        day_max=min(base_day + 2, DORMANT_SECOND_WINDOW[1]))
        amt = np.random.lognormal(cfg["amount_mean"], cfg["amount_sigma"])
        ch  = random.choice(cfg["channels"])
        fraud_rows.append(build_row(
            txn_id, cust, acc, device, ts, amt, ch,
            "dormant_to_offshore", 1,
            receiver_account_id=None, beneficiary_id=exit_bene,
        ))
        txn_id += 1

_doff_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == "dormant_to_offshore")
print(f"  Dormant→offshore   : {_doff_count:>6,} rows  ({n_offshore} accounts targeted)")
print(f"  (offshore/crypto pool: {len(exit_pool):,} beneficiaries)")

# ── Assemble fraud_df ─────────────────────────────────────────────────────────
fraud_df = pd.DataFrame(fraud_rows, columns=TXN_COLS)

# ── Validation ────────────────────────────────────────────────────────────────
unique_types = fraud_df["fraud_type"].nunique()
assert unique_types == 8, f"❌ Expected 8 fraud typologies, got {unique_types}: {fraud_df['fraud_type'].unique()}"

# Flow tracing sanity: mule_ring and layering should have non-null flow IDs
flow_types = fraud_df[fraud_df["synthetic_flow_id"].notna()]["fraud_type"].unique()
assert "mule_ring" in flow_types and "layering" in flow_types, "❌ Flow IDs missing from mule_ring or layering"

# Non-flow typologies must have null flow IDs
non_flow_types = {"ATO","smurfing","identity_fraud","dormant_ATO","dormant_smurfing","dormant_to_offshore"}
non_flow_df = fraud_df[fraud_df["fraud_type"].isin(non_flow_types)]
assert non_flow_df["synthetic_flow_id"].isna().all(), "❌ Non-flow typologies have non-null flow IDs!"

# cash_flag must be 1 for all smurfing rows
smurf_mask = fraud_df["fraud_type"].isin({"smurfing","dormant_smurfing"})
assert fraud_df.loc[smurf_mask, "cash_flag"].eq(1).all(), "❌ Smurfing rows missing cash_flag=1!"

print(f"\n✅ All validations passed")
print(f"   Total fraud rows (pre-rebalance): {len(fraud_df):,}")
print(f"   Unique typologies               : {unique_types}  {sorted(fraud_df['fraud_type'].unique())}")
print(f"   Flow-tagged rows                : {fraud_df['synthetic_flow_id'].notna().sum():,}")
print(f"   cash_flag=1 smurfing rows       : {fraud_df.loc[smurf_mask,'cash_flag'].sum():,}")
print()
fraud_df["fraud_type"].value_counts()


  Mule rings         :  4,800 rows  (60 rings × 10 accs × 8 txns)
  Layering chains    :  2,700 rows  (900 chains × 3 hops)
  ATO                :  2,000 rows
  Smurfing           :  2,415 rows  (400 groups)
  Identity fraud     :    122 rows
  Dormant ATO        :    915 rows  (255 accounts targeted)
  Dormant smurfing   :    463 rows  (106 accounts targeted)
  Dormant→offshore   :    246 rows  (127 accounts targeted)
  (offshore/crypto pool: 572 beneficiaries)

✅ All validations passed
   Total fraud rows (pre-rebalance): 13,661
   Unique typologies               : 8  ['ATO', 'dormant_ATO', 'dormant_smurfing', 'dormant_to_offshore', 'identity_fraud', 'layering', 'mule_ring', 'smurfing']
   Flow-tagged rows                : 7,500
   cash_flag=1 smurfing rows       : 2,878



fraud_type
mule_ring              4800
layering               2700
smurfing               2415
ATO                    2000
dormant_ATO             915
dormant_smurfing        463
dormant_to_offshore     246
identity_fraud          122
Name: count, dtype: int64

In [10]:
# ── CELL 7: Merge & Rebalance to Target Fraud Rate ───────────────────────

combined      = pd.concat([legit_df, fraud_df], ignore_index=True)
current_fraud = int(combined["label"].sum())
current_total = len(combined)
current_rate  = current_fraud / current_total

print(f"Before rebalancing: {current_rate:.4f} fraud rate ({current_fraud:,} / {current_total:,})")

if current_rate < TARGET_FRAUD_RATE:
    needed = int(TARGET_FRAUD_RATE * current_total / (1 - TARGET_FRAUD_RATE)) - current_fraud
    extra  = fraud_df.sample(n=needed, replace=True, random_state=42).copy()
    extra["transaction_id"] = [f"T{txn_id + i}_OS" for i in range(len(extra))]
    combined = pd.concat([combined, extra], ignore_index=True)
    print(f"Oversampled {needed:,} fraud rows to reach {TARGET_FRAUD_RATE:.0%} target.")

raw_txns = combined.sample(frac=1, random_state=42).reset_index(drop=True)
raw_txns["timestamp"] = pd.to_datetime(raw_txns["timestamp"])

final_rate = raw_txns["label"].mean()
print(f"\n✅ Final: {final_rate:.4f} fraud rate ({int(raw_txns['label'].sum()):,} / {len(raw_txns):,})")
print()
raw_txns["fraud_type"].value_counts()

Before rebalancing: 0.0321 fraud rate (13,661 / 424,992)
Oversampled 33,560 fraud rows to reach 10% target.

✅ Final: 0.1030 fraud rate (47,221 / 458,552)



fraud_type
normal                 411331
mule_ring               16540
layering                 9407
smurfing                 8395
ATO                      6879
dormant_ATO              3121
dormant_smurfing         1633
dormant_to_offshore       826
identity_fraud            420
Name: count, dtype: int64

In [11]:
# ── CELL 8: Entity Enrichment (FK Joins) ─────────────────────────────────

txns = raw_txns.copy()

# Join account features
acc_join_cols = [
    "account_id", "avg_balance", "account_open_days",
    "kyc_level", "country_risk", "income_bracket",
    "customer_risk_rating", "pep_flag", "occupation",
    "industry", "account_type",
    # Geo seed + shared identity — denormalised from customers in Notebook 1
    "home_lat", "home_lon", "home_city",
    "shared_kyc_id", "shared_phone_hash", "shared_email_hash",
]
txns = txns.merge(
    accounts[acc_join_cols].rename(columns={"account_id": "sender_account_id"}),
    on="sender_account_id", how="left"
)
print(f"After account join   : {txns.shape[1]} columns")

# Join device features
dev_cols = ["device_id", "device_age_days", "rooted_flag", "os_type", "vpn_flag", "emulator_flag"]
txns = txns.merge(devices[dev_cols], on="device_id", how="left")
print(f"After device join    : {txns.shape[1]} columns")

# Join beneficiary features (nullable FK — internal transfers get NaN)
bene_cols = ["beneficiary_id", "beneficiary_type", "beneficiary_country_risk"]
txns = txns.merge(beneficiaries[bene_cols], on="beneficiary_id", how="left")
print(f"After bene join      : {txns.shape[1]} columns")

print(f"\n✅ Entity enrichment complete: {len(txns):,} rows × {txns.shape[1]} cols")

After account join   : 34 columns
After device join    : 39 columns
After bene join      : 41 columns

✅ Entity enrichment complete: 458,552 rows × 41 cols


In [12]:
txns.head()

,transaction_id,customer_id,sender_account_id,receiver_account_id,beneficiary_id,device_id,timestamp,amount,channel,debit_credit,...,shared_kyc_id,shared_phone_hash,shared_email_hash,device_age_days,rooted_flag,os_type,vpn_flag,emulator_flag,beneficiary_type,beneficiary_country_risk
0,T442664_OS,C1141,A2951,None,B3067,D1689,2025-12-26 20:28:37,13334.64,web,debit,...,NaN,NaN,NaN,275,0,android,0,0,individual,high
1,T340270,C426,A7024,None,B4506,D3460,2025-12-21 13:27:34,147.82,mobile,debit,...,NaN,NaN,NaN,622,0,android,0,0,merchant,low
2,T352597,C1075,A7285,None,B3106,D4205,2025-11-09 07:04:43,2603.87,mobile,debit,...,NaN,NaN,EMAIL_00011,1035,0,windows,0,0,individual,low
3,T88346,C206,A1819,A2697,None,D4054,2025-11-24 22:15:16,1232.27,mobile,debit,...,NaN,NaN,NaN,467,0,android,0,0,NaN,NaN
4,T278981,C713,A5756,None,B3712,D3686,2025-11-23 07:17:14,1001.32,mobile,debit,...,KYC_00066,NaN,NaN,910,0,ios,0,0,individual,medium


In [13]:
# ── CELL 8b: IP Graph Layer ──────────────────────────────────────────────────
# Adds ip_address, ip_risk_score, geo_lat, geo_lon per transaction.
#
# IP type assignment (per transaction):
#   70% home_ip  → stable, hash-derived from customer+account IDs
#   20% roam_ip  → random public Indian IPv4 (travel / mobile data)
#   10% risk_ip  → from HIGH_RISK_IP_PREFIXES pool (Tor/proxy ranges)
#   OVERRIDE: vpn_flag=1 always forces risk_ip regardless of random draw
#
# ip_risk_score (0.0–1.0):
#   base(home=0.05 / roam=0.30 / risk=0.80)
#   + 0.15 if vpn_flag=1
#   + 0.10 if rooted_flag=1
#   + 0.10 if emulator_flag=1
#   + 0.10 if is_night=1
#   + 0.05 if country_risk="high"
#   - 0.05 if kyc_level="high"
#   capped to [0.0, 1.0]
#
# geo_lat / geo_lon:
#   home_ip  → home_lat/lon ± N(0, 0.05°)  ≈ 5 km jitter
#   roam_ip  → random Indian city ± N(0, 0.3°)
#   risk_ip  → global random (outside India — Tor exit / offshore proxy)

print("Computing IP Graph Layer features...")

ROAM_CITIES = [
    (28.6139,77.2090),(19.0760,72.8777),(12.9716,77.5946),(22.5726,88.3639),
    (17.3850,78.4867),(13.0827,80.2707),(18.5204,73.8567),(23.0225,72.5714),
    (26.9124,75.7873),(21.1702,72.8311),
]

rng_ip  = np.random.default_rng(RANDOM_SEED + 99)
RISK_IPS = [
    f"{pfx}.{rng_ip.integers(1,254)}.{rng_ip.integers(1,254)}"
    for pfx in HIGH_RISK_IP_PREFIXES for _ in range(20)
]

def _home_ip(customer_id, account_id):
    h = hash(str(customer_id) + str(account_id)) & 0xFFFF
    return f"10.{(h >> 8) & 0xFF}.{h & 0xFF}.{hash(str(account_id)) & 0xFF}"

rng_geo = np.random.default_rng(RANDOM_SEED + 77)

ip_addresses, ip_risk_scores, geo_lats, geo_lons = [], [], [], []

for _, row in txns.iterrows():
    cid    = row["customer_id"]
    aid    = row["sender_account_id"]
    vpn    = int(row.get("vpn_flag",    0) or 0)
    rooted = int(row.get("rooted_flag", 0) or 0)
    emu    = int(row.get("emulator_flag",0) or 0)
    night  = int(row.get("is_night",    0) or 0)
    crisk  = row.get("country_risk", "low")  or "low"
    kyc    = row.get("kyc_level",    "medium") or "medium"

    # ── IP type ────────────────────────────────────────────────────────────
    if vpn == 1:
        ip_type = "risk"
    else:
        r = float(rng_geo.random())
        ip_type = ("home" if r < IP_HOME_FRAC
                   else "roam" if r < IP_HOME_FRAC + IP_ROAM_FRAC
                   else "risk")

    # ── IP address ─────────────────────────────────────────────────────────
    if ip_type == "home":
        ip = _home_ip(cid, aid)
    elif ip_type == "roam":
        ip = (f"{int(rng_geo.integers(100,223))}."
              f"{int(rng_geo.integers(0,255))}."
              f"{int(rng_geo.integers(0,255))}."
              f"{int(rng_geo.integers(1,254))}")
    else:
        ip = RISK_IPS[int(rng_geo.integers(0, len(RISK_IPS)))]
    ip_addresses.append(ip)

    # ── ip_risk_score ───────────────────────────────────────────────────────
    base  = (IP_HOME_BASE_SCORE if ip_type == "home"
             else IP_ROAM_BASE_SCORE if ip_type == "roam"
             else IP_RISK_BASE_SCORE)
    score = base
    score += 0.15 if vpn    else 0
    score += 0.10 if rooted else 0
    score += 0.10 if emu    else 0
    score += 0.10 if night  else 0
    score += 0.05 if crisk == "high" else 0
    score -= 0.05 if kyc   == "high" else 0
    ip_risk_scores.append(round(min(max(score, 0.0), 1.0), 4))

    # ── Geo coordinates ─────────────────────────────────────────────────────
    if ip_type == "home":
        if cust_geo_map is not None and cid in cust_geo_map.index:
            hlat = float(cust_geo_map.loc[cid, "home_lat"])
            hlon = float(cust_geo_map.loc[cid, "home_lon"])
        else:
            hlat, hlon = 20.5937, 78.9629   # geographic centre of India fallback
        geo_lats.append(round(hlat + float(rng_geo.normal(0, 0.05)), 6))
        geo_lons.append(round(hlon + float(rng_geo.normal(0, 0.05)), 6))
    elif ip_type == "roam":
        city = ROAM_CITIES[int(rng_geo.integers(0, len(ROAM_CITIES)))]
        geo_lats.append(round(city[0] + float(rng_geo.normal(0, 0.3)), 6))
        geo_lons.append(round(city[1] + float(rng_geo.normal(0, 0.3)), 6))
    else:
        geo_lats.append(round(float(rng_geo.uniform(-60, 70)), 6))
        geo_lons.append(round(float(rng_geo.uniform(-180, 180)), 6))

txns["ip_address"]    = ip_addresses
txns["ip_risk_score"] = ip_risk_scores
txns["geo_lat"]       = geo_lats
txns["geo_lon"]       = geo_lons

print(f"✅ IP Graph Layer added: {txns.shape[1]} cols")
print(f"   home IPs   (score ≤ 0.20): {(txns['ip_risk_score'] <= 0.20).sum():,}")
print(f"   roam IPs   (0.20–0.69)   : {((txns['ip_risk_score'] > 0.20) & (txns['ip_risk_score'] < 0.70)).sum():,}")
print(f"   risk IPs   (score ≥ 0.70): {(txns['ip_risk_score'] >= 0.70).sum():,}")
print(f"   Avg ip_risk_score: {txns['ip_risk_score'].mean():.4f}")
txns[["transaction_id","ip_address","ip_risk_score","geo_lat","geo_lon"]].head(4)


Computing IP Graph Layer features...
✅ IP Graph Layer added: 45 cols
   home IPs   (score ≤ 0.20): 295,975
   roam IPs   (0.20–0.69)   : 84,899
   risk IPs   (score ≥ 0.70): 77,678
   Avg ip_risk_score: 0.2321


,transaction_id,ip_address,ip_risk_score,geo_lat,geo_lon
0,T442664_OS,91.108.91.203,0.80,46.472276,-128.320448
1,T340270,190.61.6.199,0.30,13.023940,80.486617
2,T352597,185.220.3.136,0.80,54.522884,-108.804683
3,T88346,10.160.31.54,0.05,19.006924,72.835977


In [14]:
# ── CELL 9: Temporal Features ────────────────────────────────────────────
# FIX: Ensure timestamp is parsed before deriving all temporal columns.
# week_of_year uses isocalendar() which returns UInt32 — cast to int explicitly.

txns["timestamp"]         = pd.to_datetime(txns["timestamp"])
txns["hour"]              = txns["timestamp"].dt.hour
txns["day_of_week"]       = txns["timestamp"].dt.dayofweek   # 0=Mon, 6=Sun
txns["date"]              = txns["timestamp"].dt.date
txns["month"]             = txns["timestamp"].dt.month
txns["week_of_year"]      = txns["timestamp"].dt.isocalendar().week.astype(int)

# is_night: 22:00–05:59 (inclusive on both ends)
txns["is_night"]          = ((txns["hour"] >= 22) | (txns["hour"] < 6)).astype(int)
txns["is_weekend"]        = (txns["day_of_week"] >= 5).astype(int)  # Sat=5, Sun=6
# is_business_hours: 09:00–17:59
txns["is_business_hours"] = ((txns["hour"] >= 9) & (txns["hour"] <= 17)).astype(int)

# Verify: is_night and is_business_hours must be mutually exclusive
overlap = ((txns["is_night"] == 1) & (txns["is_business_hours"] == 1)).sum()
assert overlap == 0, f"is_night / is_business_hours overlap: {overlap} rows!"

print(f"✅ Temporal features added: {txns.shape[1]} cols")
print(f"   Night transactions  : {txns['is_night'].sum():,} ({txns['is_night'].mean()*100:.1f}%)")
print(f"   Weekend transactions: {txns['is_weekend'].sum():,} ({txns['is_weekend'].mean()*100:.1f}%)")
print(f"   Business hours      : {txns['is_business_hours'].sum():,} ({txns['is_business_hours'].mean()*100:.1f}%)")
print(f"   is_night/biz overlap: {overlap} ✅")
print()
txns[["transaction_id","timestamp","hour","day_of_week","is_night","is_weekend","is_business_hours"]].head(4)

✅ Temporal features added: 53 cols
   Night transactions  : 69,576 (15.2%)
   Weekend transactions: 127,937 (27.9%)
   Business hours      : 261,808 (57.1%)
   is_night/biz overlap: 0 ✅



,transaction_id,timestamp,hour,day_of_week,is_night,is_weekend,is_business_hours
0,T442664_OS,2025-12-26 20:28:37,20,4,0,0,0
1,T340270,2025-12-21 13:27:34,13,6,0,1,1
2,T352597,2025-11-09 07:04:43,7,6,0,1,0
3,T88346,2025-11-24 22:15:16,22,0,1,0,0


In [15]:
# ── CELL 10: Amount & Balance Ratio Features ─────────────────────────────

txns["amount_to_balance_ratio"] = (
    txns["amount"] / txns["avg_balance"].clip(lower=1)
).round(4)

txns["is_round_amount"] = ((txns["amount"] % 1000 == 0) & (txns["amount"] >= 1000)).astype(int)

txns["log_amount"] = np.log1p(txns["amount"]).round(4)

hr_pool_set = set(high_risk_bene_pool)
txns["high_risk_beneficiary"] = txns["beneficiary_id"].isin(hr_pool_set).astype(int)

txns["account_age_days"] = (
    txns["timestamp"] - pd.to_datetime(START_DATE - timedelta(days=1)) +
    pd.to_timedelta(txns["account_open_days"], unit="D")
).dt.days.clip(lower=1)

print(f"✅ Amount & ratio features added: {txns.shape[1]} cols")
print()
txns[["amount","avg_balance","amount_to_balance_ratio","is_round_amount",
      "log_amount","account_age_days"]].describe().round(3)

✅ Amount & ratio features added: 58 cols



,amount,avg_balance,amount_to_balance_ratio,is_round_amount,log_amount,account_age_days
count,458552.000,458552.000,458552.000,458552.000,458552.000,458552.000
mean,10758.196,36390.376,0.805,0.000,8.374,1087.978
std,20959.611,46263.959,2.900,0.004,1.366,567.534
min,12.640,599.210,0.000,0.000,2.613,31.000
25%,1644.145,11280.590,0.062,0.000,7.406,606.000
50%,4340.640,21927.310,0.195,0.000,8.376,1098.000
75%,10913.925,43864.920,0.623,0.000,9.298,1574.000
max,1177712.140,663447.710,678.925,1.000,13.979,2121.000


In [16]:
# ── CELL 11: Account-Level Velocity Features (Rolling Windows) ───────────
#
# FIX (dormancy_flag bug):
# The original code used shift(1) on a DatetimeIndex inside groupby.apply,
# which shifted the INDEX values (timestamps), not the account's previous
# transaction time. This always produced NaT or nonsensical gaps → flag=0.
#
# Correct approach:
#   - Use .index.to_series() AFTER sorting to get chronological timestamps
#   - diff() gives the actual timedelta between consecutive rows
#   - Convert to days and flag if gap > 30 days
#   - The FIRST transaction per account always gets dormancy_flag=0
#     (no prior transaction to measure gap from)
#
# NOTE: rolling() windows are closed on the RIGHT (current row is INCLUDED).
# For txn_count_last_1hr, a transaction counts itself → minimum value is 1.

print("Computing account-level rolling velocity features...")

txns_sorted = txns.sort_values(["sender_account_id", "timestamp"]).copy()
txns_sorted = txns_sorted.set_index("timestamp")

def account_rolling(group):
    """Compute all account-level rolling/velocity features for one account."""
    group = group.sort_index()   # ensure chronological order within account
    amt   = group["amount"]

    # Rolling counts & sums (window is closed on right, includes current row)
    group["txn_count_last_1hr"]     = amt.rolling("1H").count().astype(int)
    group["txn_count_last_24hr"]    = amt.rolling("24H").count().astype(int)
    group["total_amount_last_24hr"] = amt.rolling("24H").sum().round(2)
    group["txn_count_last_7d"]      = amt.rolling("7D").count().astype(int)
    group["txn_count_last_30d"]     = amt.rolling("30D").count().astype(int)
    group["total_amount_last_7d"]   = amt.rolling("7D").sum().round(2)
    group["total_amount_last_30d"]  = amt.rolling("30D").sum().round(2)

    # Cumulative count (overall account activity level)
    group["txn_velocity_cumulative"] = range(1, len(group) + 1)

    # ── FIX: dormancy_flag ─────────────────────────────────────────────────
    # diff() on a DatetimeIndex gives timedelta between consecutive rows.
    # First row always gets NaT → fillna(False) → 0 (no prior activity).
    ts_series = group.index.to_series()
    gap_days  = ts_series.diff().dt.total_seconds().div(86400)
    group["dormancy_flag"] = (gap_days > 30).fillna(False).astype(int)

    return group

txns_sorted = (
    txns_sorted
    .groupby("sender_account_id", group_keys=False)
    .apply(account_rolling)
    .reset_index()
)

# ── Validation: dormancy_flag should now be non-zero for dormant accounts ─
dormant_txns = txns_sorted[txns_sorted["sender_account_id"].isin(dormant_account_set)]
dormant_flag_rate = dormant_txns["dormancy_flag"].mean()
overall_flag_rate = txns_sorted["dormancy_flag"].mean()
n_dormant_accs_flagged = dormant_txns[dormant_txns["dormancy_flag"]==1]["sender_account_id"].nunique()

print(f"✅ Account velocity features added: {txns_sorted.shape[1]} cols")
print(f"   dormancy_flag overall rate      : {overall_flag_rate:.4f} ({txns_sorted['dormancy_flag'].sum():,} rows)")
print(f"   dormancy_flag in dormant accounts: {dormant_flag_rate:.4f}")
print(f"   dormant accounts actually flagged: {n_dormant_accs_flagged:,} / {len(dormant_account_set):,}")
print()

vel_cols = ["transaction_id","sender_account_id","amount",
            "txn_count_last_1hr","txn_count_last_24hr","total_amount_last_24hr",
            "txn_count_last_7d","txn_count_last_30d","dormancy_flag"]
txns_sorted[vel_cols].head(6)

Computing account-level rolling velocity features...
✅ Account velocity features added: 67 cols
   dormancy_flag overall rate      : 0.0009 (428 rows)
   dormancy_flag in dormant accounts: 0.0298
   dormant accounts actually flagged: 423 / 425



,transaction_id,sender_account_id,amount,txn_count_last_1hr,txn_count_last_24hr,total_amount_last_24hr,txn_count_last_7d,txn_count_last_30d,dormancy_flag
0,T15,A0,1501.54,1,1,1501.54,1,1,0
1,T23,A0,1225.28,1,1,1225.28,2,2,0
2,T11,A0,161.86,1,1,161.86,3,3,0
3,T1,A0,5029.27,1,2,5191.13,3,4,0
4,T4,A0,5319.92,1,1,5319.92,4,5,0
5,T43,A0,533.87,1,1,533.87,2,6,0


In [17]:
# ── CELL 12: Customer-Level Velocity Features ────────────────────────────
# Aggregates across ALL accounts belonging to a customer.

print("Computing customer-level rolling velocity features...")

cust_sorted = txns_sorted.sort_values(["customer_id", "timestamp"]).copy()
cust_sorted = cust_sorted.set_index("timestamp")

def customer_rolling(group):
    group = group.sort_index()
    amt   = group["amount"]

    group["cust_txn_count_last_1hr"]     = amt.rolling("1H").count().astype(int)
    group["cust_txn_count_last_24hr"]    = amt.rolling("24H").count().astype(int)
    group["cust_total_amount_last_24hr"] = amt.rolling("24H").sum().round(2)
    group["cust_total_amount_last_30d"]  = amt.rolling("30D").sum().round(2)

    return group

cust_sorted = (
    cust_sorted
    .groupby("customer_id", group_keys=False)
    .apply(customer_rolling)
    .reset_index()
)

# ── Unique accounts used by customer in last 30 days ─────────────────────
cust_sorted = cust_sorted.sort_values(["customer_id", "timestamp"])

def unique_accounts_30d(group):
    group = group.set_index("timestamp").sort_index()
    result = []
    times  = group.index
    accs   = group["sender_account_id"].values
    for i, t in enumerate(times):
        cutoff = t - pd.Timedelta("30D")
        window = accs[(times >= cutoff) & (times <= t)]
        result.append(len(set(window)))
    group["cust_unique_accounts_30d"] = result
    return group.reset_index()

cust_sorted = (
    cust_sorted
    .groupby("customer_id", group_keys=False)
    .apply(unique_accounts_30d)
)

print(f"✅ Customer velocity features added: {cust_sorted.shape[1]} cols")
print()
cust_vel_cols = ["transaction_id","customer_id","sender_account_id","amount",
                 "cust_txn_count_last_1hr","cust_txn_count_last_24hr",
                 "cust_total_amount_last_24hr","cust_unique_accounts_30d"]
cust_sorted[cust_vel_cols].head(5)

Computing customer-level rolling velocity features...
✅ Customer velocity features added: 72 cols



,transaction_id,customer_id,sender_account_id,amount,cust_txn_count_last_1hr,cust_txn_count_last_24hr,cust_total_amount_last_24hr,cust_unique_accounts_30d
0,T313429,C0,A6471,7011.86,1,1,7011.86,1
1,T73104,C0,A1507,3093.91,1,2,10105.77,2
2,T313393,C0,A6471,3431.29,1,1,3431.29,2
3,T73100,C0,A1507,13628.43,1,2,17059.72,2
4,T313424,C0,A6471,1534.30,1,1,1534.30,2


In [21]:
# ── CELL 12b: Flow Tracing · Running Balance · Fraud Intensity Score ─────────
#
# ── A. Flow Tracing — time_since_origin_txn ───────────────────────────────────
#
#   HOW time_since_origin_ts IS STORED (Cell 6 — source of truth):
#
#   LAYERING (Cell 6B):
#     base_ts  = random_ts()                ← synthetic chain anchor
#     For hop i (0-indexed), i in range(chain_length):
#       ts = base_ts + timedelta(minutes = random_N * (i+1))
#       hop_number = i + 1                  ← 1, 2, 3
#       time_since_origin_ts = base_ts      ← same anchor stored for ALL hops
#
#     Therefore:  delta = (ts - base_ts).total_seconds()
#                       = random_N * (i+1) * 60
#                 hop 1: delta = N*1*60 > 0  (NOT zero — base_ts is NOT hop1's ts)
#                 hop 2: delta = N*2*60 > 0
#                 hop 3: delta = N*3*60 > 0
#     ✅ ALL layering deltas are strictly positive. No special casing needed.
#
#   MULE RING (Cell 6A):
#     ring_origin_ts = random_ts()          ← shared reference, one per ring
#     For each account in ring:
#       ts = random_ts()                    ← INDEPENDENT random timestamp
#       time_since_origin_ts = ring_origin_ts
#
#     Therefore:  delta = (ts - ring_origin_ts).total_seconds()
#                 can be NEGATIVE (ts drawn independently from ring_origin_ts)
#     ✅ Use abs() → non-negative "temporal distance from ring anchor"
#
#   time_since_origin_txn output semantics:
#     layering  : elapsed seconds from chain anchor → always > 0
#     mule_ring : absolute seconds from ring anchor → always >= 0
#     all others: NaN (NULL in CSV)
#
# ── B. Running Balance Features ───────────────────────────────────────────────
#   Per-account chronological state machine.
#   signed_amount = +amount (credit) or -amount (debit)
#   balance_before_txn[first] = avg_balance  (account seed)
#   balance_before_txn[i]     = balance_after_txn[i-1]
#   balance_after_txn         = max(0, balance_before + signed_amount)
#   cumulative_daily_balance_change = sum(signed_amount) per account per date
#
# ── C. Fraud Intensity Score (0–100) ─────────────────────────────────────────
#   raw = FIS_W_RULE    × weighted_rule_score
#       + FIS_W_LABEL   × label × 10
#       + FIS_W_BALANCE × min(amount_to_balance_ratio, 10)
#   fraud_intensity_score = min(raw, FIS_SCALE)
#   NOTE: weighted_rule_score is added in Cell 13. If this cell runs before
#   Cell 13, the column won't exist → score is NaN (correct, fills after Cell 13).

print("Computing Flow Tracing, Running Balance, and Fraud Intensity Score...")

# ── A. time_since_origin_txn ─────────────────────────────────────────────────
if "time_since_origin_ts" in cust_sorted.columns:
    cust_sorted["time_since_origin_ts"] = pd.to_datetime(
        cust_sorted["time_since_origin_ts"], errors="coerce"
    )

    has_flow     = cust_sorted["synthetic_flow_id"].notna()
    is_mule_ring = cust_sorted["fraud_type"] == "mule_ring"

    # Raw signed delta (seconds). Layering always positive. Mule ring can be negative.
    raw_delta_s = (
        cust_sorted["timestamp"] - cust_sorted["time_since_origin_ts"]
    ).dt.total_seconds().round(1)

    # Mule ring: abs() because ring_origin_ts is an independent anchor, not the
    # first-hop timestamp. Layering: raw delta is always positive, no change needed.
    flow_delta = np.where(is_mule_ring, raw_delta_s.abs(), raw_delta_s)

    cust_sorted["time_since_origin_txn"] = np.where(
        has_flow,
        flow_delta,
        np.nan          # non-flow rows → NULL in CSV
    )

    cust_sorted.drop(columns=["time_since_origin_ts"], inplace=True, errors="ignore")
else:
    cust_sorted["time_since_origin_txn"] = np.nan

# ── B. Running Balance (per-account chronological state machine) ──────────────
bal_sorted = cust_sorted.sort_values(["sender_account_id", "timestamp"]).copy()
bal_sorted["_signed"] = np.where(
    bal_sorted["debit_credit"] == "credit",
    bal_sorted["amount"],
    -bal_sorted["amount"]
)

bal_before_list, bal_after_list, cum_daily_list = [], [], []
_bal_state   = {}   # account_id → running balance
_daily_state = {}   # account_id → {date → cumulative signed_amount today}

for _, row in bal_sorted.iterrows():
    acc  = row["sender_account_id"]
    sa   = row["_signed"]
    date = row["date"]

    if acc not in _bal_state:
        _bal_state[acc]   = float(row["avg_balance"])
        _daily_state[acc] = {}

    bb = _bal_state[acc]
    ba = max(0.0, bb + sa)
    _daily_state[acc][date] = _daily_state[acc].get(date, 0.0) + sa

    bal_before_list.append(round(bb, 2))
    bal_after_list.append(round(ba, 2))
    cum_daily_list.append(round(_daily_state[acc][date], 2))
    _bal_state[acc] = ba

bal_sorted["balance_before_txn"]                 = bal_before_list
bal_sorted["balance_after_txn"]                  = bal_after_list
bal_sorted["current_balance"]                    = bal_sorted["balance_after_txn"]
bal_sorted["running_balance_txn_amount"]         = bal_sorted["_signed"]
bal_sorted["cumulative_daily_balance_change"]    = cum_daily_list
bal_sorted.drop(columns=["_signed"], inplace=True)

# ── C. Fraud Intensity Score ──────────────────────────────────────────────────
if "weighted_rule_score" in bal_sorted.columns:
    raw_fis = (
        FIS_W_RULE    * bal_sorted["weighted_rule_score"].fillna(0) +
        FIS_W_LABEL   * bal_sorted["label"] * 10 +
        FIS_W_BALANCE * bal_sorted["amount_to_balance_ratio"].fillna(0).clip(upper=10)
    )
    bal_sorted["fraud_intensity_score"] = raw_fis.clip(upper=FIS_SCALE).round(2)
else:
    bal_sorted["fraud_intensity_score"] = np.nan

# Re-sort back to customer+timestamp order (expected by downstream cells)
cust_sorted = bal_sorted.sort_values(["customer_id", "timestamp"]).reset_index(drop=True)

# ── Validation ────────────────────────────────────────────────────────────────

# 1️⃣ No negative balances after any transaction
assert (cust_sorted["balance_after_txn"] >= 0).all(), \
    "❌ Negative balance_after_txn detected!"

# 2️⃣ Layering: ALL hops (including hop 1) must have delta > 0
#    Reason: ts[hop_i] = base_ts + N*(i+1) mins, so delta = N*(i+1)*60 > 0 always.
#    base_ts is a synthetic anchor — it is NOT stored as any hop's actual timestamp.
layering_mask  = cust_sorted["fraud_type"] == "layering"
layering_delta = cust_sorted.loc[
    layering_mask & cust_sorted["time_since_origin_txn"].notna(),
    "time_since_origin_txn"
]
assert layering_delta.notna().all(), \
    "❌ Layering rows have NaN time_since_origin_txn!"
assert (layering_delta > 0).all(), \
    f"❌ Layering time_since_origin_txn must be > 0 for ALL hops! Min={layering_delta.min()}"

# 3️⃣ Mule ring: all deltas must be >= 0 (we applied abs())
mule_mask  = cust_sorted["fraud_type"] == "mule_ring"
mule_delta = cust_sorted.loc[
    mule_mask & cust_sorted["time_since_origin_txn"].notna(),
    "time_since_origin_txn"
]
assert mule_delta.notna().all(), \
    "❌ Mule ring rows have NaN time_since_origin_txn!"
assert (mule_delta >= 0).all(), \
    f"❌ Mule ring time_since_origin_txn must be >= 0! Min={mule_delta.min()}"

# 4️⃣ Mule ring structural integrity
mule_struct = cust_sorted.loc[mule_mask, ["synthetic_flow_id", "flow_depth"]]
assert mule_struct["synthetic_flow_id"].notna().all(), \
    "❌ Mule ring rows missing synthetic_flow_id!"
assert mule_struct["flow_depth"].notna().all(), \
    "❌ Mule ring rows missing flow_depth!"

# 5️⃣ Non-flow typologies must have NaN time_since_origin_txn
non_flow_types = {"ATO", "smurfing", "identity_fraud",
                  "dormant_ATO", "dormant_smurfing", "dormant_to_offshore", "normal"}
non_flow_delta = cust_sorted.loc[
    cust_sorted["fraud_type"].isin(non_flow_types),
    "time_since_origin_txn"
]
assert non_flow_delta.isna().all(), \
    "❌ Non-flow typologies have non-NaN time_since_origin_txn!"

print("✅ Flow validation passed:")
print(f"   Layering : {len(layering_delta):,} rows | delta range {layering_delta.min():.0f}s – {layering_delta.max():.0f}s (all > 0 ✓)")
print(f"   Mule ring: {len(mule_delta):,} rows   | delta range {mule_delta.min():.0f}s – {mule_delta.max():.0f}s (all ≥ 0 ✓)")
print(f"✅ All three feature groups added: {cust_sorted.shape[1]} cols")
print(f"   balance_before_txn : ₹{cust_sorted['balance_before_txn'].mean():>12,.2f}  mean")
print(f"   balance_after_txn  : ₹{cust_sorted['balance_after_txn'].mean():>12,.2f}  mean  "
      f"(min = ₹{cust_sorted['balance_after_txn'].min():,.2f})")
print(f"   Flow txns w/ delta : {cust_sorted['time_since_origin_txn'].notna().sum():,}")
if cust_sorted["fraud_intensity_score"].notna().any():
    print(f"   FIS mean (fraud)   : {cust_sorted.loc[cust_sorted['label']==1,'fraud_intensity_score'].mean():.2f}")
    print(f"   FIS mean (legit)   : {cust_sorted.loc[cust_sorted['label']==0,'fraud_intensity_score'].mean():.2f}")
else:
    print("   FIS: NaN (weighted_rule_score not yet computed — will populate after Cell 13)")

cust_sorted[["transaction_id","balance_before_txn","running_balance_txn_amount",
             "balance_after_txn","cumulative_daily_balance_change",
             "time_since_origin_txn","fraud_intensity_score"]].head(5)

Computing Flow Tracing, Running Balance, and Fraud Intensity Score...
✅ Flow validation passed:
   Layering : 0 rows | delta range nans – nans (all > 0 ✓)
   Mule ring: 0 rows   | delta range nans – nans (all ≥ 0 ✓)
✅ All three feature groups added: 78 cols
   balance_before_txn : ₹    6,538.10  mean
   balance_after_txn  : ₹    5,886.45  mean  (min = ₹0.00)
   Flow txns w/ delta : 0
   FIS: NaN (weighted_rule_score not yet computed — will populate after Cell 13)


,transaction_id,balance_before_txn,running_balance_txn_amount,balance_after_txn,cumulative_daily_balance_change,time_since_origin_txn,fraud_intensity_score
0,T313429,7811.76,-7011.86,799.90,-7011.86,NaN,NaN
1,T73104,79334.25,-3093.91,76240.34,-3093.91,NaN,NaN
2,T313393,799.90,-3431.29,0.00,-3431.29,NaN,NaN
3,T73100,76240.34,-13628.43,62611.91,-13628.43,NaN,NaN
4,T313424,0.00,-1534.30,0.00,-1534.30,NaN,NaN


In [22]:
# ── CELL 13: AML Rule Engine (50 Rules) ──────────────────────────────────

RULES = [
    # Amount-based
    ("large_cash_deposit",         3, lambda r: r["cash_flag"]==1 and r["amount"]>50_000),
    ("cash_just_below_threshold",  3, lambda r: r["cash_flag"]==1 and 8_000<=r["amount"]<=9_999),
    ("high_value_transfer",        2, lambda r: r["amount"]>100_000),
    ("micro_transaction",          1, lambda r: r["amount"]<10),
    ("round_amount_large",         1, lambda r: r.get("is_round_amount",0)==1 and r["amount"]>=10_000),

    # Time-based
    ("night_transaction",          2, lambda r: r.get("is_night",0)==1),
    ("weekend_high_value",         2, lambda r: r.get("is_weekend",0)==1 and r["amount"]>20_000),
    ("dormant_account_activation", 3, lambda r: r.get("dormancy_flag",0)==1 and r["amount"]>5_000),

    # Velocity (account-level)
    ("high_acct_velocity_1hr",     3, lambda r: r.get("txn_count_last_1hr",0)>5),
    ("high_acct_velocity_24hr",    2, lambda r: r.get("txn_count_last_24hr",0)>20),
    ("high_acct_velocity_7d",      2, lambda r: r.get("txn_count_last_7d",0)>60),
    ("high_acct_volume_24hr",      2, lambda r: r.get("total_amount_last_24hr",0)>500_000),
    ("high_acct_volume_7d",        2, lambda r: r.get("total_amount_last_7d",0)>2_000_000),
    ("amount_spike_30d",           3, lambda r: r.get("amount_zscore_30d",0)>4),
    ("rapid_burst",                3, lambda r: r.get("txn_count_last_1hr",0)>3),

    # Velocity (customer-level)
    ("high_cust_velocity_1hr",     3, lambda r: r.get("cust_txn_count_last_1hr",0)>8),
    ("high_cust_velocity_24hr",    2, lambda r: r.get("cust_txn_count_last_24hr",0)>30),
    ("high_cust_volume_24hr",      2, lambda r: r.get("cust_total_amount_last_24hr",0)>1_000_000),
    ("high_cust_volume_7d",        2, lambda r: r.get("cust_total_amount_last_7d",0)>3_000_000),

    # Account / KYC
    ("low_kyc_high_amount",        3, lambda r: r.get("kyc_level")=="low" and r["amount"]>30_000),
    ("new_account_large_txn",      3, lambda r: r.get("account_open_days",9999)<60 and r["amount"]>10_000),
    ("high_amount_to_balance",     2, lambda r: r.get("amount_to_balance_ratio",0)>3),
    ("very_high_risk_customer",    3, lambda r: r.get("customer_risk_rating")=="very_high"),
    ("pep_high_value",             3, lambda r: r.get("pep_flag",0)==1 and r["amount"]>10_000),
    ("high_risk_country_sender",   2, lambda r: r.get("country_risk")=="high"),
    ("corporate_large_cash",       2, lambda r: r.get("account_type") in ("corporate","business")
                                                 and r["cash_flag"]==1 and r["amount"]>20_000),

    # Beneficiary
    ("high_risk_beneficiary",      3, lambda r: r.get("high_risk_beneficiary",0)==1),
    ("crypto_transfer",            2, lambda r: r.get("beneficiary_type")=="crypto"),
    ("offshore_transfer",          3, lambda r: r.get("beneficiary_type")=="offshore"),
    ("high_risk_bene_country",     2, lambda r: r.get("beneficiary_country_risk")=="high"),

    # Device / channel
    ("rooted_device",              2, lambda r: r.get("rooted_flag",0)==1),
    ("vpn_proxy_detected",         2, lambda r: r.get("vpn_flag",0)==1),
    ("emulator_detected",          3, lambda r: r.get("emulator_flag",0)==1),
    ("new_device_large_txn",       1, lambda r: r.get("device_age_days",9999)<90 and r["amount"]>10_000),
    ("atm_high_withdrawal",        2, lambda r: r["channel"]=="atm" and r["amount"]>20_000),
    ("branch_night_txn",           2, lambda r: r["channel"]=="branch" and r.get("is_night",0)==1),

    # Structuring
    ("structuring_pattern",        3, lambda r: r["cash_flag"]==1 and 8_500<=r["amount"]<=9_999),
    ("multiple_small_cash",        3, lambda r: r["cash_flag"]==1
                                                 and r.get("txn_count_last_7d",0)>5 and r["amount"]<10_000),

    # Industry / occupation
    ("high_risk_industry",         1, lambda r: r.get("industry") in ("real_estate","construction","unknown")),
    ("student_high_value",         2, lambda r: r.get("occupation")=="student" and r["amount"]>50_000),
    ("unemployed_large_transfer",  2, lambda r: r.get("occupation")=="unemployed" and r["amount"]>20_000),
    ("freelancer_offshore",        2, lambda r: r.get("occupation")=="freelancer"
                                                 and r.get("beneficiary_type")=="offshore"),

    # Combined risk combos
    ("pep_crypto_transfer",        3, lambda r: r.get("pep_flag",0)==1 and r.get("beneficiary_type")=="crypto"),
    ("very_high_risk_offshore",    3, lambda r: r.get("customer_risk_rating")=="very_high"
                                                 and r.get("beneficiary_type")=="offshore"),
    ("low_kyc_offshore",           3, lambda r: r.get("kyc_level")=="low" and r.get("beneficiary_type")=="offshore"),
    ("low_income_large_txn",       2, lambda r: r.get("income_bracket")=="low" and r["amount"]>50_000),
    ("vpn_offshore",               3, lambda r: r.get("vpn_flag",0)==1 and r.get("beneficiary_type")=="offshore"),
    ("emulator_crypto",            3, lambda r: r.get("emulator_flag",0)==1 and r.get("beneficiary_type")=="crypto"),
    ("new_account_offshore",       3, lambda r: r.get("account_open_days",9999)<90
                                                 and r.get("beneficiary_type")=="offshore"),
    ("new_acct_high_cust_velocity",3, lambda r: r.get("account_open_days",9999)<90
                                                 and r.get("cust_txn_count_last_24hr",0)>10),
]

result = cust_sorted.copy()

# ── Apply each rule as a standalone 0/1 column ───────────────────────────
# severity_map drives the weighted aggregates below — no numpy matrix needed.
severity_map = {}
for rule_name, severity, condition in RULES:
    col = f"rule_{rule_name}"
    try:
        result[col] = result.apply(lambda row, c=condition: int(bool(c(row))), axis=1)
    except Exception:
        result[col] = 0
    severity_map[col] = severity

# # ── Aggregate scores derived directly from the rule columns ──────────────
# # rule_cols is used only here for the aggregation; no intermediate array.
# rule_cols = [f"rule_{n}" for n, _, _ in RULES]

# result["rule_trigger_count"]  = result[rule_cols].sum(axis=1).astype(int)

# result["max_rule_severity"]   = result[rule_cols].mul(
#     pd.Series(severity_map)                      # aligns by column name
# ).max(axis=1).astype(int)

# result["weighted_rule_score"] = result[rule_cols].mul(
#     pd.Series(severity_map)
# ).sum(axis=1).astype(int)

# ── Validation & preview ──────────────────────────────────────────────────
n_dormant_rule = result["rule_dormant_account_activation"].sum()
#total_firings  = int(result["rule_trigger_count"].sum())
print(f"✅ Rule engine complete")
# print(f"   {len(RULES)} rules | {total_firings:,} total firings | "
#       f"avg {result['rule_trigger_count'].mean():.2f} per transaction")
print(f"   rule_dormant_account_activation firings: {n_dormant_rule:,}")
print()
# result[["transaction_id","label","fraud_type","dormancy_flag",
#         "rule_dormant_account_activation",
#         "rule_trigger_count","max_rule_severity","weighted_rule_score"]].head(5)

result.head()

✅ Rule engine complete
   rule_dormant_account_activation firings: 222



,timestamp,transaction_id,customer_id,sender_account_id,receiver_account_id,beneficiary_id,device_id,amount,channel,debit_credit,...,rule_unemployed_large_transfer,rule_freelancer_offshore,rule_pep_crypto_transfer,rule_very_high_risk_offshore,rule_low_kyc_offshore,rule_low_income_large_txn,rule_vpn_offshore,rule_emulator_crypto,rule_new_account_offshore,rule_new_acct_high_cust_velocity
0,2025-09-01 14:01:04,T313429,C0,A6471,None,B473,D1143,7011.86,mobile,debit,...,0,0,0,0,0,0,0,0,0,0
1,2025-09-02 09:42:26,T73104,C0,A1507,A459,None,D43,3093.91,branch,debit,...,0,0,0,0,0,0,0,0,0,0
2,2025-09-03 13:38:26,T313393,C0,A6471,None,B1227,D1143,3431.29,mobile,debit,...,0,0,0,0,0,0,0,0,0,0
3,2025-09-04 01:42:35,T73100,C0,A1507,None,B4002,D43,13628.43,web,debit,...,0,0,0,0,0,0,0,0,0,0
4,2025-09-06 22:10:02,T313424,C0,A6471,A4745,None,D1143,1534.30,web,debit,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# ── CELL 14: Final Transactions Table Summary ─────────────────────────────

final_txns = result.copy()

print("══════════════════════════════════════════════════════════════")
print("  FINAL TRANSACTIONS TABLE SUMMARY")
print("══════════════════════════════════════════════════════════════")
print(f"  Rows             : {len(final_txns):,}")
print(f"  Columns          : {final_txns.shape[1]}")
print(f"  Fraud rate       : {final_txns['label'].mean():.4f}")
print()
print("  PK / FK columns:")
pk_fk_cols = ["transaction_id","customer_id","sender_account_id",
              "receiver_account_id","beneficiary_id","device_id"]
for col in pk_fk_cols:
    null_pct = final_txns[col].isna().mean() * 100
    print(f"    {col:<30} {null_pct:.1f}% null")

print()
print("  XOR check (receiver_account_id XOR beneficiary_id):")
both    = (final_txns["receiver_account_id"].notna() & final_txns["beneficiary_id"].notna()).sum()
neither = (final_txns["receiver_account_id"].isna()  & final_txns["beneficiary_id"].isna()).sum()
print(f"    Rows with BOTH set   : {both}  {'✅' if both==0 else '❌'}")
print(f"    Rows with NEITHER set: {neither}  {'✅' if neither==0 else '❌'}")

print()
print("  Dormancy stats:")
print(f"    dormancy_flag = 1 rows   : {final_txns['dormancy_flag'].sum():,} "
      f"({final_txns['dormancy_flag'].mean()*100:.2f}%)")
print(f"    Forced dormant accounts  : {len(dormant_account_set):,} / {len(all_account_ids):,} "
      f"({len(dormant_account_set)/len(all_account_ids):.1%})")

print()
print("  Feature groups:")
base_cols  = [c for c in TXN_COLS if c in final_txns.columns]
enrich     = ["avg_balance","account_open_days","kyc_level","pep_flag",
              "device_age_days","rooted_flag","vpn_flag","emulator_flag",
              "beneficiary_type","beneficiary_country_risk"]
temporal   = ["hour","day_of_week","is_night","is_weekend","is_business_hours","month"]
ratio      = ["amount_to_balance_ratio","is_round_amount","log_amount",
              "high_risk_beneficiary","account_age_days"]
acct_vel   = [c for c in final_txns.columns
              if c.startswith("txn_") or c.startswith("total_amount_last")
              or c == "dormancy_flag" or c == "txn_velocity_cumulative"]
cust_vel   = [c for c in final_txns.columns if c.startswith("cust_")]
rule_agg   = ["rule_trigger_count","max_rule_severity","weighted_rule_score"]
rule_flags = [c for c in final_txns.columns if c.startswith("rule_") and c not in rule_agg]

for group_name, cols in [
    ("Core transaction",  base_cols),
    ("Entity enrichment", [c for c in enrich if c in final_txns.columns]),
    ("Temporal",          temporal),
    ("Amount / ratios",   ratio),
    ("Account velocity",  acct_vel),
    ("Customer velocity", cust_vel),
    ("Rule aggregates",   rule_agg),
    ("Rule flags",        rule_flags),
]:
    print(f"    {group_name:<20} : {len(cols)} columns")

══════════════════════════════════════════════════════════════
  FINAL TRANSACTIONS TABLE SUMMARY
══════════════════════════════════════════════════════════════
  Rows             : 458,552
  Columns          : 128
  Fraud rate       : 0.1030

  PK / FK columns:
    transaction_id                 0.0% null
    customer_id                    0.0% null
    sender_account_id              0.0% null
    receiver_account_id            62.0% null
    beneficiary_id                 38.0% null
    device_id                      0.0% null

  XOR check (receiver_account_id XOR beneficiary_id):
    Rows with BOTH set   : 0  ✅
    Rows with NEITHER set: 0  ✅

  Dormancy stats:
    dormancy_flag = 1 rows   : 428 (0.09%)
    Forced dormant accounts  : 425 / 8,500 (5.0%)

  Feature groups:
    Core transaction     : 17 columns
    Entity enrichment    : 10 columns
    Temporal             : 6 columns
    Amount / ratios      : 5 columns
    Account velocity     : 9 columns
    Customer velocity    : 5

In [24]:
print("DONE with the Process")

DONE with the Process


In [25]:
# ── CELL 15: Fraud Typology Breakdown ────────────────────────────────────

print("══════════════════════════════════════════════════════════════")
print("  FRAUD TYPOLOGY SUMMARY")
print("══════════════════════════════════════════════════════════════")

fraud_only = final_txns[final_txns["label"] == 1]

print(f"  Total rows      : {len(final_txns):,}")
print(f"  Fraud rows      : {len(fraud_only):,} ({len(fraud_only)/len(final_txns)*100:.2f}%)")
print(f"  Legit rows      : {len(final_txns)-len(fraud_only):,}")
print()

breakdown = final_txns[final_txns["label"]==1]["fraud_type"].value_counts()
for ftype, cnt in breakdown.items():
    pct = cnt / len(fraud_only) * 100
    bar = "█" * int(pct / 2)
    dormant_tag = " [DORMANT]" if ftype.startswith("dormant_") else ""
    print(f"  {ftype:<22} {cnt:>7,}  ({pct:>5.1f}%)  {bar}{dormant_tag}")

print()
print("  Velocity by fraud type (mean values):")
vel_summary = final_txns.groupby("fraud_type").agg(
    avg_amount=("amount", "mean"),
    avg_1hr_txns=("txn_count_last_1hr", "mean"),
    avg_24hr_txns=("txn_count_last_24hr", "mean"),
    avg_night=("is_night", "mean"),
    avg_high_risk_bene=("high_risk_beneficiary", "mean"),
    avg_dormancy=("dormancy_flag", "mean"),
).round(4)
print(vel_summary.to_string())

print()
print("  Dormant account fraud cross-tab (dormancy_flag × fraud_type):")
dormant_fraud_types = ["dormant_ATO", "dormant_smurfing", "dormant_to_offshore"]
dormant_fraud_df = final_txns[final_txns["fraud_type"].isin(dormant_fraud_types)]
if len(dormant_fraud_df) > 0:
    cross = dormant_fraud_df.groupby("fraud_type").agg(
        rows=("transaction_id", "count"),
        unique_accounts=("sender_account_id", "nunique"),
        dormancy_flag_rate=("dormancy_flag", "mean"),
        avg_amount=("amount", "mean"),
        pct_night=("is_night", "mean"),
    ).round(3)
    print(cross.to_string())
else:
    print("  No dormant fraud rows found — check DORMANT_ACCOUNT_FRAC and thresholds.")

══════════════════════════════════════════════════════════════
  FRAUD TYPOLOGY SUMMARY
══════════════════════════════════════════════════════════════
  Total rows      : 458,552
  Fraud rows      : 47,221 (10.30%)
  Legit rows      : 411,331

  mule_ring               16,540  ( 35.0%)  █████████████████
  layering                 9,407  ( 19.9%)  █████████
  smurfing                 8,395  ( 17.8%)  ████████
  ATO                      6,879  ( 14.6%)  ███████
  dormant_ATO              3,121  (  6.6%)  ███ [DORMANT]
  dormant_smurfing         1,633  (  3.5%)  █ [DORMANT]
  dormant_to_offshore        826  (  1.7%)   [DORMANT]
  identity_fraud             420  (  0.9%)  

  Velocity by fraud type (mean values):
                      avg_amount  avg_1hr_txns  avg_24hr_txns  avg_night  avg_high_risk_bene  avg_dormancy
fraud_type                                                                                                
ATO                   49310.8884        2.6055         3.1303     

## QC Checks

In [26]:
# ── CELL 16: PK / FK Integrity Validation ────────────────────────────────

print("══════════════════════════════════════════════════════════════")
print("  FK INTEGRITY REPORT")
print("══════════════════════════════════════════════════════════════")

checks = {
    # Core PK / FK integrity
    "transaction_id is unique"                      : final_txns["transaction_id"].is_unique,
    "customer_id → customers FK (no orphans)"       : final_txns["customer_id"].isin(customers["customer_id"]).all(),
    "sender_account_id → accounts FK (no orphans)"  : final_txns["sender_account_id"].isin(accounts["account_id"]).all(),
    "device_id → devices FK (no orphans)"           : final_txns["device_id"].isin(devices["device_id"]).all(),
    "receiver_account_id XOR beneficiary_id"        : (
        (final_txns["receiver_account_id"].notna() & final_txns["beneficiary_id"].notna()).sum() == 0 and
        (final_txns["receiver_account_id"].isna()  & final_txns["beneficiary_id"].isna()).sum() == 0
    ),
    # Feature sanity checks
    "dormancy_flag has non-zero values"             : final_txns["dormancy_flag"].sum() > 0,
    "is_night/is_business_hours mutually exclusive" : ((final_txns["is_night"]==1) & (final_txns["is_business_hours"]==1)).sum() == 0,
    "txn_count_last_1hr >= 1 for all rows"          : (final_txns["txn_count_last_1hr"] >= 1).all(),
    # Dormant fraud typology integrity
    "dormant_ATO only targets dormant accounts"     : final_txns.loc[
        final_txns["fraud_type"]=="dormant_ATO", "sender_account_id"
    ].isin(dormant_account_set).all() if (final_txns["fraud_type"]=="dormant_ATO").any() else True,
    "dormant_smurfing only targets dormant accounts": final_txns.loc[
        final_txns["fraud_type"]=="dormant_smurfing", "sender_account_id"
    ].isin(dormant_account_set).all() if (final_txns["fraud_type"]=="dormant_smurfing").any() else True,
    "dormant_to_offshore only targets dormant accs" : final_txns.loc[
        final_txns["fraud_type"]=="dormant_to_offshore", "sender_account_id"
    ].isin(dormant_account_set).all() if (final_txns["fraud_type"]=="dormant_to_offshore").any() else True,
    "dormant_smurfing cash_flag=1 always"           : (
        final_txns.loc[final_txns["fraud_type"]=="dormant_smurfing", "cash_flag"] == 1
    ).all() if (final_txns["fraud_type"]=="dormant_smurfing").any() else True,
    "dormant_to_offshore uses offshore/crypto benes": (
        final_txns.loc[
            final_txns["fraud_type"]=="dormant_to_offshore", "beneficiary_id"
        ].isin(set(exit_pool)).all()
    ) if (final_txns["fraud_type"]=="dormant_to_offshore").any() else True,
}

all_ok = True
for check, passed in checks.items():
    icon = "✅" if passed else "❌"
    print(f"  {icon}  {check}")
    if not passed:
        all_ok = False

print()
print(f"  Overall: {'✅ ALL CHECKS PASSED' if all_ok else '❌ FAILURES DETECTED'}")

══════════════════════════════════════════════════════════════
  FK INTEGRITY REPORT
══════════════════════════════════════════════════════════════
  ✅  transaction_id is unique
  ✅  customer_id → customers FK (no orphans)
  ✅  sender_account_id → accounts FK (no orphans)
  ✅  device_id → devices FK (no orphans)
  ✅  receiver_account_id XOR beneficiary_id
  ✅  dormancy_flag has non-zero values
  ✅  is_night/is_business_hours mutually exclusive
  ✅  txn_count_last_1hr >= 1 for all rows
  ✅  dormant_ATO only targets dormant accounts
  ✅  dormant_smurfing only targets dormant accounts
  ✅  dormant_to_offshore only targets dormant accs
  ✅  dormant_smurfing cash_flag=1 always
  ✅  dormant_to_offshore uses offshore/crypto benes

  Overall: ✅ ALL CHECKS PASSED


In [27]:
# ── CELL 17: Complete Feature Catalogue ──────────────────────────────────

FULL_CATALOGUE = {
    # ── CORE TRANSACTION FIELDS ───────────────────────────────────────────
    "transaction_id":           "PK. Unique transaction identifier. Format: T0, T1, …",
    "customer_id":              "FK → customers.customer_id. Denormalised for single-key joins.",
    "sender_account_id":        "FK → accounts.account_id. Account that initiated the payment.",
    "receiver_account_id":      "FK → accounts.account_id. NULLABLE. Set for internal transfers only.",
    "beneficiary_id":           "FK → beneficiaries.beneficiary_id. NULLABLE. Set for external transfers only.",
    "device_id":                "FK → devices.device_id. Device used to initiate transaction.",
    "timestamp":                "Full datetime of transaction (UTC).",
    "amount":                   "Transaction amount in INR. Lognormally distributed.",
    "channel":                  "Channel: mobile | web | branch | atm.",
    "debit_credit":             "Always 'debit' — sender perspective.",
    "transaction_type":         "Payment instrument: UPI | IMPS | wallet_transfer | NEFT | RTGS | online_transfer | cash_deposit | cash_withdrawal | DD | balance_enquiry.",
    "cash_flag":                "1 = physical cash involved (cash_deposit, cash_withdrawal, DD).",
    "fraud_type":               "Ground truth typology: normal | mule_ring | layering | ATO | smurfing | identity_fraud.",
    "label":                    "Target variable. 0 = legitimate, 1 = fraud.",

    # ── ENTITY ENRICHMENT (from FK joins) ─────────────────────────────────
    "avg_balance":              "From accounts. Average account balance (INR). Denominator for ratio features.",
    "account_open_days":        "From accounts. Days since account opened. <60 days = elevated identity fraud risk.",
    "kyc_level":                "From accounts/customers. KYC depth: low | medium | high.",
    "country_risk":             "From accounts/customers. Home jurisdiction risk: low | medium | high.",
    "income_bracket":           "From accounts/customers. Declared income: low | medium | high.",
    "customer_risk_rating":     "From accounts/customers. CDD rating: low | medium | high | very_high.",
    "pep_flag":                 "From accounts/customers. 1 = Politically Exposed Person.",
    "occupation":               "From accounts/customers. Declared occupation.",
    "industry":                 "From accounts/customers. Business sector.",
    "account_type":             "From accounts/customers. Account type: retail | corporate | savings | current | business.",
    "device_age_days":          "From devices. Age of device in days.",
    "rooted_flag":              "From devices. 1 = rooted/jailbroken device.",
    "os_type":                  "From devices. android | ios | windows | unknown.",
    "vpn_flag":                 "From devices. 1 = VPN/proxy detected.",
    "emulator_flag":            "From devices. 1 = emulated environment (not physical device).",
    "beneficiary_type":         "From beneficiaries. individual | merchant | crypto | offshore.",
    "beneficiary_country_risk": "From beneficiaries. Jurisdiction risk: low | medium | high.",

    # ── TEMPORAL FEATURES ─────────────────────────────────────────────────
    "hour":                     "Hour of day (0–23). Fraud peaks at 0–4 and 20–23.",
    "day_of_week":              "Day of week (0=Mon … 6=Sun).",
    "date":                     "Calendar date of transaction.",
    "month":                    "Calendar month (1–12).",
    "week_of_year":             "ISO week number (1–53).",
    "is_night":                 "1 = transaction between 22:00 and 05:59.",
    "is_weekend":               "1 = Saturday or Sunday.",
    "is_business_hours":        "1 = transaction between 09:00 and 17:00.",

    # ── AMOUNT & RATIO FEATURES ────────────────────────────────────────────
    "amount_to_balance_ratio":  "amount / avg_balance (clipped at 1). >1 means txn exceeds typical balance.",
    "is_round_amount":          "1 = amount is a round multiple of ₹1,000 and ≥ ₹1,000.",
    "log_amount":               "log1p(amount). Log-transformed for ML models.",
    "high_risk_beneficiary":    "1 = beneficiary_id is in the high-risk pool (crypto/offshore/high-country-risk).",
    "account_age_days":         "Account age in days at the time of this transaction (dynamic, not static).",

    # ── ACCOUNT-LEVEL VELOCITY (per sender_account_id) ────────────────────
    "txn_count_last_1hr":       "ACCOUNT: Number of transactions sent in the last 1 hour.",
    "txn_count_last_24hr":      "ACCOUNT: Number of transactions sent in the last 24 hours.",
    "total_amount_last_24hr":   "ACCOUNT: Total INR sent in the last 24 hours.",
    "txn_count_last_7d":        "ACCOUNT: Transaction count in the last 7 days.",
    "txn_count_last_30d":       "ACCOUNT: Transaction count in the last 30 days.",
    "total_amount_last_7d":     "ACCOUNT: Total INR sent in the last 7 days.",
    "total_amount_last_30d":    "ACCOUNT: Total INR sent in the last 30 days.",
    "amount_zscore_30d":        "ACCOUNT: Z-score of this amount vs 30-day rolling mean. >3 = anomalous spike.",
    "dormancy_flag":            "ACCOUNT: 1 = >30 days since last transaction from this account.",
    "txn_velocity_cumulative":  "ACCOUNT: Cumulative count of all transactions from this account.",

    # ── CUSTOMER-LEVEL VELOCITY (per customer_id, across ALL accounts) ────
    "cust_txn_count_last_1hr":     "CUSTOMER: Transactions across all accounts in last 1 hour.",
    "cust_txn_count_last_24hr":    "CUSTOMER: Transactions across all accounts in last 24 hours.",
    "cust_total_amount_last_24hr": "CUSTOMER: Total INR across all accounts in last 24 hours.",
    "cust_txn_count_last_7d":      "CUSTOMER: Transaction count across all accounts in last 7 days.",
    "cust_total_amount_last_7d":   "CUSTOMER: Total INR across all accounts in last 7 days.",
    "cust_txn_count_last_30d":     "CUSTOMER: Transaction count across all accounts in last 30 days.",
    "cust_total_amount_last_30d":  "CUSTOMER: Total INR across all accounts in last 30 days.",
    "cust_unique_accounts_30d":    "CUSTOMER: Number of unique accounts used (cumulative proxy).",


    # ── IP GRAPH LAYER ────────────────────────────────────────────────────────
    "ip_address":               "Per-transaction IP. 70% home (stable, hash-derived), 20% roaming, 10% high-risk proxy. Always risk_ip when vpn_flag=1.",
    "ip_risk_score":            "Float 0.0–1.0. Base: home=0.05, roam=0.30, risk=0.80. Modifiers: +0.15 vpn, +0.10 rooted/emulator/night each, +0.05 high-country-risk, −0.05 high-kyc. Capped [0,1].",
    "geo_lat":                  "Latitude of transaction origin. home_ip→home_lat±N(0,0.05°); roam_ip→Indian city±0.3°; risk_ip→global random.",
    "geo_lon":                  "Longitude of transaction origin. Same logic as geo_lat.",

    # ── FLOW TRACING ──────────────────────────────────────────────────────────
    "synthetic_flow_id":        "Chain/ring identifier. FLOW_00001… (mule_ring 0–9999, layering 10000+). NULL for ATO, smurfing, identity_fraud, dormant_* typologies.",
    "flow_depth":               "Total hops in this flow (accounts_per_ring for mule_ring; chain_length for layering). NULL for non-flow typologies.",
    "hop_number":               "Position in flow: 1=origin, flow_depth=final. NULL for non-flow typologies.",
    "time_since_origin_txn":    "Seconds from hop_number=1 to this transaction. 0 for origin hop. NULL for non-flow typologies.",

    # ── RUNNING BALANCE ───────────────────────────────────────────────────────
    "balance_before_txn":       "Account balance immediately before this transaction. Initialised to avg_balance on first txn; carried forward as balance_after_txn thereafter. INR.",
    "balance_after_txn":        "Account balance after this transaction = max(0, balance_before_txn + signed_amount). Clamped at 0. INR.",
    "current_balance":          "Alias for balance_after_txn. Post-transaction account balance. INR.",
    "running_balance_txn_amount": "Signed amount applied to balance. −amount for debits, +amount for credits. Captures direction of balance change. INR.",
    "cumulative_daily_balance_change": "Sum of signed_amounts for this account on the same calendar date up to this row. Resets at day boundary per account. INR.",

    # ── FRAUD INTENSITY SCORE ─────────────────────────────────────────────────
    "fraud_intensity_score":    "Composite risk score 0–100. = min(0.5×weighted_rule_score + 0.3×label×10 + 0.2×min(amount_to_balance_ratio,10), 100). Bands: 0–20 very low, 21–40 low, 41–60 medium, 61–80 high, 81–100 critical.",

    # ── SHARED IDENTITY (relationship graph) ──────────────────────────────────
    "shared_kyc_id":            "Shared KYC document group (e.g. KYC_00042). NULL=unique identity. Non-NULL=multiple accounts share same KYC doc — synthetic identity / mule ring signal.",
    "shared_phone_hash":        "Shared phone number group. NULL=unique. Non-NULL=same phone across customers — mule recruitment / SIM-swap signal.",
    "shared_email_hash":        "Shared email group. NULL=unique. Non-NULL=same email across customer records — identity farm signal.",

    # ── GEO SEED (from customers, via accounts join) ───────────────────────────
    "home_city":                "Customer home city name. Seeds per-transaction geo coordinates.",
    "home_lat":                 "Customer home city latitude. Jittered per transaction for home IPs.",
    "home_lon":                 "Customer home city longitude. Jittered per transaction for home IPs.",

    # ── RULE ENGINE AGGREGATES ─────────────────────────────────────────────
    "rule_trigger_count":       "Total AML rules triggered by this transaction. Higher = more suspicious.",
    "max_rule_severity":        "Severity (1–3) of the most severe rule triggered. 0 if none.",
    "weighted_rule_score":      "Sum of severities of all triggered rules. Overall suspicion weight.",
}

cat_df = pd.DataFrame([
    {"column_name": k, "description": v,
     "is_pk_fk": "yes" if k in ("transaction_id","customer_id","sender_account_id",
                                 "receiver_account_id","beneficiary_id","device_id") else "no"}
    for k, v in FULL_CATALOGUE.items()
])
cat_df.to_csv(OUTPUT_DIR / "transaction_feature_catalogue.csv", index=False)
print(f"✅ Feature catalogue saved: {len(cat_df)} features documented")
print()
cat_df

✅ Feature catalogue saved: 85 features documented



,column_name,description,is_pk_fk
0,transaction_id,"PK. Unique transaction identifier. Format: T0,...",yes
1,customer_id,FK → customers.customer_id. Denormalised for s...,yes
2,sender_account_id,FK → accounts.account_id. Account that initiat...,yes
3,receiver_account_id,FK → accounts.account_id. NULLABLE. Set for in...,yes
4,beneficiary_id,FK → beneficiaries.beneficiary_id. NULLABLE. S...,yes
...,...,...,...
80,home_lat,Customer home city latitude. Jittered per tran...,no
81,home_lon,Customer home city longitude. Jittered per tra...,no
82,rule_trigger_count,Total AML rules triggered by this transaction....,no
83,max_rule_severity,Severity (1–3) of the most severe rule trigger...,no


In [28]:
cat_df.to_excel(OUTPUT_DIR / "transaction_feature_catalogue.xlsx", index=False)

In [29]:
# ── CELL 18: Save Final Transactions ─────────────────────────────────────

final_txns.to_csv(OUTPUT_DIR / "transactions_additional_features.csv", index=False)

size_kb = (OUTPUT_DIR / "transactions_additional_features.csv").stat().st_size / 1024
print(f"✅ transactions.csv saved")
print(f"   {len(final_txns):,} rows × {final_txns.shape[1]} cols | {size_kb:,.0f} KB")
print()
print("Quick join recipes:")
print()
print("  # Transactions + customer profile:")
print("  transactions.merge(customers, on='customer_id')")
print()
print("  # Transactions + account details:")
print("  transactions.merge(accounts, left_on='sender_account_id', right_on='account_id')")
print()
print("  # External transfers only + beneficiary info:")
print("  transactions[transactions['beneficiary_id'].notna()].merge(beneficiaries, on='beneficiary_id')")
print()
print(f"  dormant_accs = {sorted(list(dormant_account_set))[:5]}... ({len(dormant_account_set)} total)")
print()
print("All outputs in ./outputs/:")
for f in sorted(OUTPUT_DIR.glob("*.csv")):
    print(f"  📄 {f.name}")

✅ transactions.csv saved
   458,552 rows × 128 cols | 226,404 KB

Quick join recipes:

  # Transactions + customer profile:
  transactions.merge(customers, on='customer_id')

  # Transactions + account details:
  transactions.merge(accounts, left_on='sender_account_id', right_on='account_id')

  # External transfers only + beneficiary info:
  transactions[transactions['beneficiary_id'].notna()].merge(beneficiaries, on='beneficiary_id')

  dormant_accs = ['A1', 'A1002', 'A1010', 'A1034', 'A1042']... (425 total)

All outputs in ./outputs/:
  📄 accounts.csv
  📄 beneficiaries.csv
  📄 customer_feature_catalogue.csv
  📄 customers.csv
  📄 devices.csv
  📄 transaction_feature_catalogue.csv
  📄 transactions.csv
  📄 transactions_additional_features.csv
